**Coding Steps for Designed Deep Neural Network Model:**
* Step - 1: Import Libraries.
* Step - 2: Define the required functions.
* Step - 3: Dataset Exploration of the pre - extracted features and labels.
    * Step - 3a : Initalize the required folders to read features and labels for each game.
    * Step - 3b : Identify the size of each feature array (.npy format) and labels (.json format).
* Step - 4: Preparation of Dataset.
    * Step - 4a : Generate the Train, Test and Valid datasets for further using it for implementing different pooling techniques.
    * Step - 4b : Splitting the datasets into required Inputs and Target.
* Step - 5: Designing the Deep Learning Model.
    * Step - 5a : Design a model and implement one by one Pooling techniques : Max Pooling, Average Pooling, Mixed Pooling, NetVLAD pooling.
    * Step - 5b : After this pooling layer, stack a fully connected layer with a dropout layer (having 60% probability) that predicts whether in a frame action is happening or not.
    * Step - 5c : Compile the model and print the model summary.
    * Step - 5d : Train the model.
    * Step - 5e : Evaluate the model.
* Step - 6: Plot train and validation accuracy and loss curves for the choosen epochs.
* Step - 7: Perform prediction of labels from the test features.
* Step - 8: Calculating important metrics for performance evaluation.
    * Confusion matrix, Classification report, Precision, Recall, F1 score, Accuracy.
* Step - 9: Plotting Confusion matrix, ROC curve, AUC score.

# **Step - 1: Import Libraries**



In [ ]:
# For navigating through different folders which are present in the operating system.
import os

# For handling json files (In this project, Labels are in json format)
import json

# For performing numerical computations
import numpy as np
from collections import Counter

# For supressing the output:
np.set_printoptions(suppress = True)

# For developing deep learning models
import tensorflow as tf
from tensorflow import keras

# For kernal regualizar:
from keras.regularizers import l2

# For visualization
import matplotlib.pyplot as plt
import seaborn as sns

# For measuring the accuracy of the models
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score,f1_score,accuracy_score 
from sklearn.metrics import roc_curve, roc_auc_score

# **Step - 2: Define the required functions.**

In [ ]:
#For loading pre - extracted C3D features
def ReadFeatures_c3d(gamefolder):
    C3D_1 = np.load(os.path.join(gamefolder,"1_C3D_PCA512.npy"))
    C3D_2 = np.load(os.path.join(gamefolder,"2_C3D_PCA512.npy"))
    
    return C3D_1, C3D_2

#For loading pre - extracted I3D features
def ReadFeatures_i3d(gamefolder):
    I3D_1 = np.load(os.path.join(gamefolder,"1_I3D_PCA512.npy"))
    I3D_2 = np.load(os.path.join(gamefolder,"2_I3D_PCA512.npy"))
    
    return I3D_1, I3D_2

#For loading pre - extracted ResNET features
def ReadFeatures_resnet(gamefolder):
    ResNET_1 = np.load(os.path.join(gamefolder,"1_ResNET_PCA512.npy"))
    ResNET_2 = np.load(os.path.join(gamefolder,"2_ResNET_PCA512.npy"))
    
    return ResNET_1, ResNET_2

#For loading Labels having various annotations 
def ReadLabels(gamefolder):
    return json.load(open(os.path.join(gamefolder,"Labels.json")))

#For extracting two important annotations Gametime and Label 
def Extract_gameTime_Label(extracted_labels_json):
    # Extract the annotations in annotations variable.
    annotations = extracted_labels_json['annotations']

    # Create an empty list where we can store the extracted values from each key.
    gameTime = []
    label = []

    # Develop a for loop to extract gameTime and label from annotations.
    for annotation in annotations:
        gameTime.append(annotation['gameTime'])
        label.append(annotation['label'])
        
    return gameTime, label

#For extracting half, minutes and seconds from Gametime
#Also, assigning new label for a particular frame. 
def Extract_half_mins_secs(gameTime,dim_label_1half,dim_label_2half):
    label_1half = np.zeros((dim_label_1half,1))
    label_2half = np.zeros((dim_label_2half,1))

    for i in range(len(gameTime)):
        half = gameTime[i][0]
        mins = gameTime[i][4:6]
        secs = gameTime[i][7:9]
        if int(half) == 1:
            frame_pos1 = ((int(mins) * 120) + (int(secs) * 2)) - 1
            label_1half[frame_pos1] = 1
        if int(half) == 2:
            frame_pos2 = ((int(mins) * 120) + (int(secs) * 2)) - 1
            label_2half[frame_pos2] = 1
    
    return label_1half, label_2half

#For generating one complete game using C3D features
def generate_games_c3d(list_games):
    C3D_1, C3D_2 = ReadFeatures_c3d(list_games)
    
    extracted_labels_json = ReadLabels(list_games)
    gameTime, label = Extract_gameTime_Label(extracted_labels_json)
    label_1half, label_2half = Extract_half_mins_secs(gameTime,np.shape(C3D_1)[0],np.shape(C3D_2)[0])
    
    C3D_1 = np.hstack((C3D_1, label_1half))
    C3D_2 = np.hstack((C3D_2, label_2half))
    
    return np.vstack((C3D_1, C3D_2))

#For generating one complete game using I3D features
def generate_games_i3d(list_games):
    I3D_1, I3D_2 = ReadFeatures_i3d(list_games)
    
    extracted_labels_json = ReadLabels(list_games)
    gameTime, label = Extract_gameTime_Label(extracted_labels_json)
    label_1half, label_2half = Extract_half_mins_secs(gameTime,np.shape(I3D_1)[0],np.shape(I3D_2)[0])
    
    I3D_1 = np.hstack((I3D_1, label_1half))
    I3D_2 = np.hstack((I3D_2, label_2half))
    
    return np.vstack((I3D_1, I3D_2))

#For generating one complete game using ResNET features
def generate_games_resnet(list_games):
    ResNET_1, ResNET_2 = ReadFeatures_resnet(list_games)
    
    extracted_labels_json = ReadLabels(list_games)
    gameTime, label = Extract_gameTime_Label(extracted_labels_json)
    label_1half, label_2half = Extract_half_mins_secs(gameTime,np.shape(ResNET_1)[0],np.shape(ResNET_2)[0])
    
    ResNET_1 = np.hstack((ResNET_1, label_1half))
    ResNET_2 = np.hstack((ResNET_2, label_2half))
    
    return np.vstack((ResNET_1, ResNET_2))

### Link to the Dataset:
#### Features: https://drive.google.com/drive/folders/1fYtz4rK2gkMTav4_JjZmA0ER42-rV1_s
#### Labels: https://drive.google.com/drive/folders/1ffjX0QU1MdPArsg-Ejm4ACp_CCa4lmFK

# **Step - 3: Dataset Exploration of the pre - extracted features and labels.**
## Step - 3a: Initalize the required folders to read features and labels for each game.

In [ ]:
Train_folder1 = "/kaggle/input/england-premier-leauge/Dataset/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley"
Train_folder2 = "/kaggle/input/england-premier-leauge/Dataset/2015-02-21 - 18-00 Crystal Palace 1 - 2 Arsenal"
Train_folder3 = "/kaggle/input/england-premier-leauge/Dataset/2015-02-21 - 18-00 Swansea 2 - 1 Manchester United"
Train_folder4 = "/kaggle/input/england-premier-leauge/Dataset/2015-02-22 - 19-15 Southampton 0 - 2 Liverpool"
Test_folder = "/kaggle/input/england-premier-leauge/Dataset/2015-05-17 - 18-00 Manchester United 1 - 1 Arsenal"
Valid_folder = "/kaggle/input/england-premier-leauge/Dataset/2015-04-11 - 19-30 Burnley 0 - 1 Arsenal"

## Step - 3b: Identify the size of each feature array (.npy format) and labels (.json format).

In [ ]:
ResNET_1, ResNET_2 = ReadFeatures_resnet(Train_folder1)
print("For Game 1: (Using ResNET Approach)")
print("1st Half: (Number of Frames x Number of Features) = ",np.shape(ResNET_1))
print("2nd Half: (Number of Frames x Number of Features) = ",np.shape(ResNET_2))

In [ ]:
C3D_1, C3D_2 = ReadFeatures_c3d(Train_folder1)
print("For Game 1: (Using C3D Approach)")
print("1st Half: (Number of Frames x Number of Features) = ",np.shape(C3D_1))
print("2nd Half: (Number of Frames x Number of Features) = ",np.shape(C3D_2))

In [ ]:
I3D_1, I3D_2 = ReadFeatures_i3d(Train_folder1)
print("For Game 1: (Using I3D Approach)")
print("1st Half: (Number of Frames x Number of Features) = ",np.shape(I3D_1))
print("2nd Half: (Number of Frames x Number of Features) = ",np.shape(I3D_2))

In [ ]:
extracted_labels_json = ReadLabels(Train_folder1)
# Extract the annotations in annotations variable.
annotations = extracted_labels_json['annotations']

# Create an empty list where we can store the extracted values from each key.
gameTime = []
label = []
print("Labels for Game 1: Information about the actions: ")
for annotation in annotations:
    gameTime = annotation['gameTime']
    label = annotation['label']
    print(f"Game Time: {gameTime} and Label: {label}")

In [ ]:
ResNET_1, ResNET_2 = ReadFeatures_resnet(Train_folder2)
print("For Game 2: (Using ResNET Approach)")
print("1st Half: (Number of Frames x Number of Features) = ",np.shape(ResNET_1))
print("2nd Half: (Number of Frames x Number of Features) = ",np.shape(ResNET_2))

In [ ]:
C3D_1, C3D_2 = ReadFeatures_c3d(Train_folder2)
print("For Game 2: (Using C3D Approach)")
print("1st Half: (Number of Frames x Number of Features) = ",np.shape(C3D_1))
print("2nd Half: (Number of Frames x Number of Features) = ",np.shape(C3D_2))

In [ ]:
I3D_1, I3D_2 = ReadFeatures_i3d(Train_folder2)
print("For Game 2: (Using I3D Approach)")
print("1st Half: (Number of Frames x Number of Features) = ",np.shape(I3D_1))
print("2nd Half: (Number of Frames x Number of Features) = ",np.shape(I3D_2))

In [ ]:
extracted_labels_json = ReadLabels(Train_folder2)
# Extract the annotations in annotations variable.
annotations = extracted_labels_json['annotations']

# Create an empty list where we can store the extracted values from each key.
gameTime = []
label = []
print("Labels for Game 2: Information about the actions: ")
for annotation in annotations:
    gameTime = annotation['gameTime']
    label = annotation['label']
    print(f"Game Time: {gameTime} and Label: {label}")

In [ ]:
ResNET_1, ResNET_2 = ReadFeatures_resnet(Train_folder3)
print("For Game 3: (Using ResNET Approach)")
print("1st Half: (Number of Frames x Number of Features) = ",np.shape(ResNET_1))
print("2nd Half: (Number of Frames x Number of Features) = ",np.shape(ResNET_2))

In [ ]:
C3D_1, C3D_2 = ReadFeatures_c3d(Train_folder3)
print("For Game 3: (Using C3D Approach)")
print("1st Half: (Number of Frames x Number of Features) = ",np.shape(C3D_1))
print("2nd Half: (Number of Frames x Number of Features) = ",np.shape(C3D_2))

In [ ]:
I3D_1, I3D_2 = ReadFeatures_i3d(Train_folder3)
print("For Game 3: (Using I3D Approach)")
print("1st Half: (Number of Frames x Number of Features) = ",np.shape(I3D_1))
print("2nd Half: (Number of Frames x Number of Features) = ",np.shape(I3D_2))

In [ ]:
extracted_labels_json = ReadLabels(Train_folder3)
# Extract the annotations in annotations variable.
annotations = extracted_labels_json['annotations']

# Create an empty list where we can store the extracted values from each key.
gameTime = []
label = []
print("Labels for Game 3: Information about the actions: ")
for annotation in annotations:
    gameTime = annotation['gameTime']
    label = annotation['label']
    print(f"Game Time: {gameTime} and Label: {label}")

In [ ]:
ResNET_1, ResNET_2 = ReadFeatures_resnet(Train_folder4)
print("For Game 4: (Using ResNET Approach)")
print("1st Half: (Number of Frames x Number of Features) = ",np.shape(ResNET_1))
print("2nd Half: (Number of Frames x Number of Features) = ",np.shape(ResNET_2))

In [ ]:
C3D_1, C3D_2 = ReadFeatures_c3d(Train_folder4)
print("For Game 4: (Using C3D Approach)")
print("1st Half: (Number of Frames x Number of Features) = ",np.shape(C3D_1))
print("2nd Half: (Number of Frames x Number of Features) = ",np.shape(C3D_2))

In [ ]:
I3D_1, I3D_2 = ReadFeatures_i3d(Train_folder4)
print("For Game 4: (Using I3D Approach)")
print("1st Half: (Number of Frames x Number of Features) = ",np.shape(I3D_1))
print("2nd Half: (Number of Frames x Number of Features) = ",np.shape(I3D_2))

In [ ]:
extracted_labels_json = ReadLabels(Train_folder4)
# Extract the annotations in annotations variable.
annotations = extracted_labels_json['annotations']

# Create an empty list where we can store the extracted values from each key.
gameTime = []
label = []
print("Labels for Game 4: Information about the actions: ")
for annotation in annotations:
    gameTime = annotation['gameTime']
    label = annotation['label']
    print(f"Game Time: {gameTime} and Label: {label}")

In [ ]:
ResNET_1, ResNET_2 = ReadFeatures_resnet(Test_folder)
print("For Game 5: (Using ResNET Approach)")
print("1st Half: (Number of Frames x Number of Features) = ",np.shape(ResNET_1))
print("2nd Half: (Number of Frames x Number of Features) = ",np.shape(ResNET_2))

In [ ]:
C3D_1, C3D_2 = ReadFeatures_c3d(Test_folder)
print("For Game 5: (Using C3D Approach)")
print("1st Half: (Number of Frames x Number of Features) = ",np.shape(C3D_1))
print("2nd Half: (Number of Frames x Number of Features) = ",np.shape(C3D_2))

In [ ]:
I3D_1, I3D_2 = ReadFeatures_i3d(Test_folder)
print("For Game 5: (Using I3D Approach)")
print("1st Half: (Number of Frames x Number of Features) = ",np.shape(I3D_1))
print("2nd Half: (Number of Frames x Number of Features) = ",np.shape(I3D_2))

In [ ]:
extracted_labels_json = ReadLabels(Test_folder)
# Extract the annotations in annotations variable.
annotations = extracted_labels_json['annotations']

# Create an empty list where we can store the extracted values from each key.
gameTime = []
label = []
print("Labels for Game 5: Information about the actions: ")
for annotation in annotations:
    gameTime = annotation['gameTime']
    label = annotation['label']
    print(f"Game Time: {gameTime} and Label: {label}")

In [ ]:
ResNET_1, ResNET_2 = ReadFeatures_resnet(Valid_folder)
print("For Game 6: (Using ResNET Approach)")
print("1st Half: (Number of Frames x Number of Features) = ",np.shape(ResNET_1))
print("2nd Half: (Number of Frames x Number of Features) = ",np.shape(ResNET_2))

In [ ]:
C3D_1, C3D_2 = ReadFeatures_c3d(Valid_folder)
print("For Game 6: (Using C3D Approach)")
print("1st Half: (Number of Frames x Number of Features) = ",np.shape(C3D_1))
print("2nd Half: (Number of Frames x Number of Features) = ",np.shape(C3D_2))

In [ ]:
I3D_1, I3D_2 = ReadFeatures_i3d(Valid_folder)
print("For Game 6: (Using I3D Approach)")
print("1st Half: (Number of Frames x Number of Features) = ",np.shape(I3D_1))
print("2nd Half: (Number of Frames x Number of Features) = ",np.shape(I3D_2))

In [ ]:
extracted_labels_json = ReadLabels(Valid_folder)
# Extract the annotations in annotations variable.
annotations = extracted_labels_json['annotations']

# Create an empty list where we can store the extracted values from each key.
gameTime = []
label = []
print("Labels for Game 6: Information about the actions: ")
for annotation in annotations:
    gameTime = annotation['gameTime']
    label = annotation['label']
    print(f"Game Time: {gameTime} and Label: {label}")

# **Step - 4: Prepration of Datasets (Train, Test and Valid).**
## Step - 4a: Generate the Train, Test and Valid datasets for further using it for implementing different pooling techniques.
## Using ResNET Features and Labels

In [ ]:
Train_game1_resnet = generate_games_resnet(Train_folder1)
print("For Train Game 1: ")
print("(Total Number of Frames x Number of Features) = ",np.shape(Train_game1_resnet))

In [ ]:
Train_game2_resnet = generate_games_resnet(Train_folder2)
print("For Train Game 2: ")
print("(Total Number of Frames x Number of Features) = ",np.shape(Train_game2_resnet))

In [ ]:
Train_game3_resnet = generate_games_resnet(Train_folder3)
print("For Train Game 3: ")
print("(Total Number of Frames x Number of Features) = ",np.shape(Train_game3_resnet))

In [ ]:
Train_game4_resnet = generate_games_resnet(Train_folder4)
print("For Train Game 4: ")
print("(Total Number of Frames x Number of Features) = ",np.shape(Train_game4_resnet))

In [ ]:
Train_games_resnet = np.vstack((Train_game1_resnet, Train_game2_resnet,Train_game3_resnet,Train_game4_resnet))
print("Train Games: ")
print("(Total Number of Frames x Number of Features) = ",np.shape(Train_games_resnet))

In [ ]:
Test_game_resnet = generate_games_resnet(Test_folder)
print("Test Game: ")
print("(Total Number of Frames x Number of Features) = ",np.shape(Test_game_resnet))

In [ ]:
Valid_game_resnet = generate_games_resnet(Valid_folder)
print("Valid Game: ")
print("(Total Number of Frames x Number of Features) = ",np.shape(Valid_game_resnet))

## Using C3D Features and Labels

In [ ]:
Train_game1_c3d = generate_games_c3d(Train_folder1)
print("For Train Game 1: ")
print("(Total Number of Frames x Number of Features) = ",np.shape(Train_game1_c3d))

In [ ]:
Train_game2_c3d = generate_games_c3d(Train_folder2)
print("For Train Game 2: ")
print("(Total Number of Frames x Number of Features) = ",np.shape(Train_game2_c3d))

In [ ]:
Train_game3_c3d = generate_games_c3d(Train_folder3)
print("For Train Game 3: ")
print("(Total Number of Frames x Number of Features) = ",np.shape(Train_game3_c3d))

In [ ]:
Train_game4_c3d = generate_games_c3d(Train_folder4)
print("For Train Game 4: ")
print("(Total Number of Frames x Number of Features) = ",np.shape(Train_game4_c3d))

In [ ]:
Train_games_c3d = np.vstack((Train_game1_c3d, Train_game2_c3d,Train_game3_c3d,Train_game4_c3d))
print("Train Games: ")
print("(Total Number of Frames x Number of Features) = ",np.shape(Train_games_c3d))

In [ ]:
Test_game_c3d = generate_games_c3d(Test_folder)
print("Test Game: ")
print("(Total Number of Frames x Number of Features) = ",np.shape(Test_game_c3d))

In [ ]:
Valid_game_c3d = generate_games_c3d(Valid_folder)
print("Valid Game: ")
print("(Total Number of Frames x Number of Features) = ",np.shape(Valid_game_c3d))

## Using I3D Features and Labels

In [ ]:
Train_game1_i3d = generate_games_i3d(Train_folder1)
print("For Train Game 1: ")
print("(Total Number of Frames x Number of Features) = ",np.shape(Train_game1_i3d))

In [ ]:
Train_game2_i3d = generate_games_i3d(Train_folder2)
print("For Train Game 2: ")
print("(Total Number of Frames x Number of Features) = ",np.shape(Train_game2_i3d))

In [ ]:
Train_game3_i3d = generate_games_i3d(Train_folder3)
print("For Train Game 3: ")
print("(Total Number of Frames x Number of Features) = ",np.shape(Train_game3_i3d))

In [ ]:
Train_game4_i3d = generate_games_i3d(Train_folder4)
print("For Train Game 4: ")
print("(Total Number of Frames x Number of Features) = ",np.shape(Train_game4_i3d))

In [ ]:
Train_games_i3d = np.vstack((Train_game1_i3d, Train_game2_i3d,Train_game3_i3d,Train_game4_i3d))
print("Train Games: ")
print("(Total Number of Frames x Number of Features) = ",np.shape(Train_games_i3d))

In [ ]:
Test_game_i3d = generate_games_i3d(Test_folder)
print("Test Game: ")
print("(Total Number of Frames x Number of Features) = ",np.shape(Test_game_i3d))

In [ ]:
Valid_game_i3d = generate_games_i3d(Valid_folder)
print("Valid Game: ")
print("(Total Number of Frames x Number of Features) = ",np.shape(Valid_game_i3d))

## Step - 4b: Splitting the generate datasets into required Inputs(Features) and Target(Assigned Labels). 
## Using ResNET Approach

In [ ]:
X_train_resnet = Train_games_resnet[:,0:Train_games_resnet.shape[1] - 1]
Y_train_resnet = Train_games_resnet[:,-1]
X_test_resnet = Test_game_resnet[:,0:Test_game_resnet.shape[1] - 1]
Y_test_resnet = Test_game_resnet[:,-1]
X_valid_resnet = Valid_game_resnet[:,0:Valid_game_resnet.shape[1] - 1]
Y_valid_resnet = Valid_game_resnet[:,-1]

print("Using ResNET Approach")
print("X_train = ",np.shape(X_train_resnet))
print("Y_train = ",np.shape(Y_train_resnet))
print("X_test = ",np.shape(X_test_resnet))
print("Y_test = ",np.shape(Y_test_resnet))
print("X_valid = ",np.shape(X_valid_resnet))
print("Y_valid = ",np.shape(Y_valid_resnet))
print("Count of Action Labels")
print("Y_train labels: {}".format(Counter(Y_train_resnet)))
print("Y_test labels: {}".format(Counter(Y_test_resnet)))
print("Y_valid labels: {} ".format(Counter(Y_valid_resnet)))

## Using C3D Approach

In [ ]:
X_train_c3d = Train_games_c3d[:,0:Train_games_c3d.shape[1] - 1]
Y_train_c3d = Train_games_c3d[:,-1]
X_test_c3d = Test_game_c3d[:,0:Test_game_c3d.shape[1] - 1]
Y_test_c3d = Test_game_c3d[:,-1]
X_valid_c3d = Valid_game_c3d[:,0:Valid_game_c3d.shape[1] - 1]
Y_valid_c3d = Valid_game_c3d[:,-1]

print("Using C3D Approach")
print("X_train = ",np.shape(X_train_c3d))
print("Y_train = ",np.shape(Y_train_c3d))
print("X_test = ",np.shape(X_test_c3d))
print("Y_test = ",np.shape(Y_test_c3d))
print("X_valid = ",np.shape(X_valid_c3d))
print("Y_valid = ",np.shape(Y_valid_c3d))
print("Count of Action Labels")
print("Y_train labels: {}".format(Counter(Y_train_c3d)))
print("Y_test labels: {}".format(Counter(Y_test_c3d)))
print("Y_valid labels: {}".format(Counter(Y_valid_c3d)))

## Using I3D Approach 

In [ ]:
X_train_i3d = Train_games_i3d[:,0:Train_games_i3d.shape[1] - 1]
Y_train_i3d = Train_games_i3d[:,-1]
X_test_i3d = Test_game_i3d[:,0:Test_game_i3d.shape[1] - 1]
Y_test_i3d = Test_game_i3d[:,-1]
X_valid_i3d = Valid_game_i3d[:,0:Valid_game_i3d.shape[1] - 1]
Y_valid_i3d = Valid_game_i3d[:,-1]

print("Using I3D Approach")
print("X_train = ",np.shape(X_train_i3d))
print("Y_train = ",np.shape(Y_train_i3d))
print("X_test = ",np.shape(X_test_i3d))
print("Y_test = ",np.shape(Y_test_i3d))
print("X_valid = ",np.shape(X_valid_i3d))
print("Y_valid = ",np.shape(Y_valid_i3d))
print("Count of Action Labels")
print("Y_train labels: {}".format(Counter(Y_train_i3d)))
print("Y_test labels: {}".format(Counter(Y_test_i3d)))
print("Y_valid labels: {}".format(Counter(Y_valid_i3d)))

# Using ResNET Approach
# Step - 5: Designing the Deep Learning Model.
### Step - 5a: Design a model that performs Max Pooling.
### Step - 5b: After this pooling layer, stack a fuly connected layer with a dropout layer (having 60% probability) that predicts whether in a frame action is happening or not.
### Step - 5c : Compile the model and print the model summary.

In [ ]:
# Number of classes in the target variable
NB_CLASSES=1

# Create a sequencial model in Keras
model_max_resnet = tf.keras.models.Sequential(name = 'Deep_Neural_Network_Model_Max-Pool1D_ResNET')

# Add a reshaping layer
model_max_resnet.add(tf.keras.layers.Reshape((1, 512),name = 'Reshaping'))
          
# Pooling using Max Pooling 1D:
model_max_resnet.add(keras.layers.MaxPool1D(pool_size = 2,strides = 1,padding = "same", name = 'Max-Pool1D-Layer'))

# Flattening the pooled features
model_max_resnet.add(keras.layers.Flatten(name = 'Flatten-Layer'))

# Add the first hidden layer
model_max_resnet.add(keras.layers.Dense(200,                    #Number of hidden nodes
                             kernel_regularizer = l2(0.01),
                             name='Hidden-Layer-1', #Logical name
                             activation='sigmoid'))    #activation function

# Add the second hidden layer
model_max_resnet.add(keras.layers.Dense(500,                    #Number of hidden nodes
                             name='Hidden-Layer-2', #Logical name
                             activation='sigmoid'))    #activation function

# Add the third hidden layer
model_max_resnet.add(keras.layers.Dense(200,                    #Number of hidden nodes
                             kernel_regularizer = l2(0.01),
                             name='Hidden-Layer-3', #Logical name
                             activation='sigmoid'))    #activation function

# Adding a Dropout layer of probability of 60%.
model_max_resnet.add(keras.layers.Dropout(0.6, name = 'Dropout-Layer'))

# Add an Output layer with sigmoid activation
model_max_resnet.add(keras.layers.Dense(NB_CLASSES,
                             name='Output-Layer',
                             activation='sigmoid'))

#Compile the model with loss & metrics
model_max_resnet.compile(optimizer = 'adam', loss = 'binary_crossentropy',
              metrics = ['accuracy'])

model_max_resnet.build(input_shape = (None,1,512))

#Print the model meta-data
model_max_resnet.summary()

In [ ]:
# Diagram of the Model Architecture
keras.utils.plot_model(model_max_resnet,to_file='model_max_resnet.png', show_shapes=True, show_layer_activations=True )

### Step - 5d: Train the model.

In [ ]:
#Make it verbose so we can see the progress
VERBOSE=1

#Set Batch size
BATCH_SIZE=120

#Set number of epochs
EPOCHS=25

print("\nTraining Progress:\n------------------------------------")

# Reshaping Train and Valid features so that it satisfies the fit function requirements. 
X_train_reshape_resnet = tf.reshape(X_train_resnet, [X_train_resnet.shape[0], 1, X_train_resnet.shape[1]])
X_valid_reshape_resnet = tf.reshape(X_valid_resnet, [X_valid_resnet.shape[0], 1, X_valid_resnet.shape[1]])

# Assigning weights to labels since there is an imbalanced in dataset labels.
# This will avoid overfitting as well as the model would not be bias.
weights_assigned = {0:1,1:892}

#Fit the model. This will perform the entire training cycle, including
#forward propagation, loss computation, backward propagation.
#Execute for the specified batch sizes and epoch
#Perform validation after each epoch.

history_max_resnet = model_max_resnet.fit(X_train_reshape_resnet,
          Y_train_resnet,
          class_weight = weights_assigned,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_data = (X_valid_reshape_resnet, Y_valid_resnet),
          verbose = VERBOSE)

### Step - 5e : Evaluate the model.

In [ ]:
#Evaluate the model against the test dataset and print results
print("\nEvaluation against Test Dataset (Using ResNET Approach - Max Pooling) :\n------------------------------------")
X_test_reshape_resnet = tf.reshape(X_test_resnet, [X_test_resnet.shape[0], 1,X_test_resnet.shape[1]])
model_max_resnet.evaluate(X_test_reshape_resnet,Y_test_resnet)

### Step - 6: Plot train and validation accuracy and loss curves for the choosen epochs.

In [ ]:
plt.plot(history_max_resnet.history['accuracy'],'r-o', label = 'train')
plt.plot(history_max_resnet.history['val_accuracy'],'b-o', label = 'validation')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.suptitle("Train Accuracy vs Validation Accuracy")
plt.title("Using ResNET Approach - Max Pooling", loc='center')
plt.legend()
plt.savefig("accuracy_max_resnet.png",dpi = 96)
plt.show()

In [ ]:
plt.plot(history_max_resnet.history['loss'],'r-o', label = 'train')
plt.plot(history_max_resnet.history['val_loss'],'g-o', label = 'validation')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.suptitle("Train Loss vs Validation Loss")
plt.title("Using ResNET Approach - Max Pooling", loc='center')
plt.legend()
plt.savefig("loss_max_resnet.png",dpi = 96)
plt.show()

### Step - 7: Perform prediction of labels from the test features.

In [ ]:
Y_pred_resnet_maxpool = model_max_resnet.predict(X_test_resnet)
Y_pred_resnet_maxpool

In [ ]:
Y_pred_resnet_maxpool_round2 = Y_pred_resnet_maxpool.round(decimals=2)
Y_pred_resnet_maxpool_round2

In [ ]:
Y_pred_resnet_maxpool_round0 = Y_pred_resnet_maxpool.round(decimals=0)
Y_pred_resnet_maxpool_round0

### Step - 8: Calculating important metrics.
#### Confusion matrix, Classification report, Precision, Recall, F1 score, Accuracy.

In [ ]:
print("Using ResNET Approach - Max Pooling")
print("Classification Report:")
print(classification_report(Y_test_resnet, Y_pred_resnet_maxpool_round0))
print("Confusion Matrix: ")
print(confusion_matrix(Y_test_resnet, Y_pred_resnet_maxpool_round0))
print('Precision Score : ',(precision_score(Y_test_resnet, Y_pred_resnet_maxpool_round0, average = 'macro')))
print('Recall Score : ',(recall_score(Y_test_resnet, Y_pred_resnet_maxpool_round0, average = 'macro')))
print('F-1 Score : ',(f1_score(Y_test_resnet, Y_pred_resnet_maxpool_round0, average = 'macro')))
print("Accuracy:", accuracy_score(Y_test_resnet, Y_pred_resnet_maxpool_round0))

### Step - 9: Plotting Confusion matrix, ROC curve, AUC score.

In [ ]:
cm_resnet_maxpool = confusion_matrix(Y_test_resnet, Y_pred_resnet_maxpool_round0)

plt.figure(figsize = (5,5))
sns.heatmap(data = cm_resnet_maxpool, linewidths = .5, annot = True, cmap = 'Blues',fmt=".0f")
plt.xlabel('Predicted label')
plt.ylabel('Actual label')
plt.suptitle('Confusion matrix',size = 15)
plt.title("Using ResNET Approach - Max Pooling", loc='center')
plt.savefig("cm_max_resnet.png",dpi = 96)
plt.show()

In [ ]:
fpr_resnet_maxpool, tpr_resnet_maxpool, tr_resnet_max = roc_curve(Y_test_resnet, np.array(Y_pred_resnet_maxpool_round2))
auc_resnet_maxpool = roc_auc_score(Y_test_resnet, Y_pred_resnet_maxpool_round2)

plt.plot(fpr_resnet_maxpool, tpr_resnet_maxpool, 'b-o', label = 'AUC = %0.4f'%auc_resnet_maxpool)
plt.plot(fpr_resnet_maxpool,fpr_resnet_maxpool,linestyle = '--',color = 'k')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.suptitle('ROC curve',size = 15)
plt.title("Using ResNET Approach - Max Pooling", loc='center')
plt.legend()
plt.savefig("roc_auc_max_resnet.png",dpi = 96)
plt.show()

# Using C3D Approach
# Step - 5: Designing the Deep Learning Model.
### Step - 5a: Design a model that performs Max Pooling.
### Step - 5b: After this pooling layer, stack a fuly connected layer with a dropout layer (having 60% probability) that predicts whether in a frame action is happening or not.
### Step - 5c : Compile the model and print the model summary.

In [ ]:
# Number of classes in the target variable
NB_CLASSES=1

# Create a sequencial model in Keras
model_max_c3d = tf.keras.models.Sequential(name = 'Deep_Neural_Network_Model_Max-Pool1D_C3D')

# Add a reshaping layer
model_max_c3d.add(tf.keras.layers.Reshape((1, 512),name = 'Reshaping'))
          
# Pooling using Max Pooling 1D:
model_max_c3d.add(keras.layers.MaxPool1D(pool_size = 2,strides = 1,padding = "same", name = 'Max-Pool1D-Layer'))

# Flattening the pooled features
model_max_c3d.add(keras.layers.Flatten(name = 'Flatten-Layer'))

# Add the first hidden layer
model_max_c3d.add(keras.layers.Dense(200,                    #Number of hidden nodes
                             kernel_regularizer = l2(0.01),
                             name='Hidden-Layer-1', #Logical name
                             activation='sigmoid'))    #activation function

# Add the second hidden layer
model_max_c3d.add(keras.layers.Dense(500,                    #Number of hidden nodes
                             name='Hidden-Layer-2', #Logical name
                             activation='sigmoid'))    #activation function

# Add the third hidden layer
model_max_c3d.add(keras.layers.Dense(200,                    #Number of hidden nodes
                             kernel_regularizer = l2(0.01),
                             name='Hidden-Layer-3', #Logical name
                             activation='sigmoid'))    #activation function

# Adding a Dropout layer of probability of 60%.
model_max_c3d.add(keras.layers.Dropout(0.6, name = 'Dropout-Layer'))

# Add an Output layer with sigmoid activation
model_max_c3d.add(keras.layers.Dense(NB_CLASSES,
                             name='Output-Layer',
                             activation='sigmoid'))

#Compile the model with loss & metrics
model_max_c3d.compile(optimizer = 'adam', loss = 'binary_crossentropy',
              metrics = ['accuracy'])

model_max_c3d.build(input_shape = (None,1,512))

#Print the model meta-data
model_max_c3d.summary()

In [ ]:
# Diagram of the Model Architecture
keras.utils.plot_model(model_max_c3d,to_file='model_max_c3d.png', show_shapes=True, show_layer_activations=True )

### Step - 5d: Train the model.

In [ ]:
#Make it verbose so we can see the progress
VERBOSE=1

#Set Batch size
BATCH_SIZE=120

#Set number of epochs
EPOCHS=25

print("\nTraining Progress:\n------------------------------------")

# Reshaping Train and Valid features so that it satisfies the fit function requirements. 
X_train_reshape_c3d = tf.reshape(X_train_c3d, [X_train_c3d.shape[0], 1, X_train_c3d.shape[1]])
X_valid_reshape_c3d = tf.reshape(X_valid_c3d, [X_valid_c3d.shape[0], 1, X_valid_c3d.shape[1]])

# Assigning weights to labels since there is an imbalanced in dataset labels.
# This will avoid overfitting as well as the model would not be bias.
weights_assigned = {0:1,1:892}

#Fit the model. This will perform the entire training cycle, including
#forward propagation, loss computation, backward propagation.
#Execute for the specified batch sizes and epoch
#Perform validation after each epoch.

history_max_c3d = model_max_c3d.fit(X_train_reshape_c3d,
          Y_train_c3d,
          class_weight = weights_assigned,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_data = (X_valid_reshape_c3d, Y_valid_c3d),
          verbose = VERBOSE)

### Step - 5e : Evaluate the model.

In [ ]:
#Evaluate the model against the test dataset and print results
print("\nEvaluation against Test Dataset (Using C3D Approach - Max Pooling) :\n------------------------------------")
X_test_reshape_c3d = tf.reshape(X_test_c3d, [X_test_c3d.shape[0], 1,X_test_c3d.shape[1]])
model_max_c3d.evaluate(X_test_reshape_c3d,Y_test_c3d)

### Step - 6: Plot train and validation accuracy and loss curves for the choosen epochs.

In [ ]:
plt.plot(history_max_c3d.history['accuracy'],'r-o', label = 'train')
plt.plot(history_max_c3d.history['val_accuracy'],'b-o', label = 'validation')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.suptitle("Train Accuracy vs Validation Accuracy")
plt.title("Using C3D Approach - Max Pooling", loc='center')
plt.legend()
plt.savefig("accuracy_max_c3d.png",dpi = 96)
plt.show()

In [ ]:
plt.plot(history_max_c3d.history['loss'],'r-o', label = 'train')
plt.plot(history_max_c3d.history['val_loss'],'g-o', label = 'validation')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.suptitle("Train Loss vs Validation Loss")
plt.title("Using C3D Approach - Max Pooling", loc='center')
plt.legend()
plt.savefig("loss_max_c3d.png",dpi = 96)
plt.show()

### Step - 7: Perform prediction of labels from the test features.

In [ ]:
Y_pred_c3d_maxpool = model_max_c3d.predict(X_test_c3d)
Y_pred_c3d_maxpool

In [ ]:
Y_pred_c3d_maxpool_round2 = Y_pred_c3d_maxpool.round(decimals=2)
Y_pred_c3d_maxpool_round2

In [ ]:
Y_pred_c3d_maxpool_round0 = Y_pred_c3d_maxpool.round(decimals=0)
Y_pred_c3d_maxpool_round0

### Step - 8: Calculating important metrics.
#### Confusion matrix, Classification report, Precision, Recall, F1 score, Accuracy.

In [ ]:
print("Using C3D Approach - Max Pooling")
print("Classification Report:")
print(classification_report(Y_test_c3d, Y_pred_c3d_maxpool_round0))
print("Confusion Matrix: ")
print(confusion_matrix(Y_test_c3d, Y_pred_c3d_maxpool_round0))
print('Precision Score : ',(precision_score(Y_test_c3d, Y_pred_c3d_maxpool_round0, average = 'macro')))
print('Recall Score : ',(recall_score(Y_test_c3d, Y_pred_c3d_maxpool_round0, average = 'macro')))
print('F-1 Score : ',(f1_score(Y_test_c3d, Y_pred_c3d_maxpool_round0, average = 'macro')))
print("Accuracy:", accuracy_score(Y_test_c3d, Y_pred_c3d_maxpool_round0))

### Step - 9: Plotting Confusion matrix, ROC curve, AUC score.

In [ ]:
cm_c3d_maxpool = confusion_matrix(Y_test_c3d, Y_pred_c3d_maxpool_round0)

plt.figure(figsize = (5,5))
sns.heatmap(data = cm_c3d_maxpool, linewidths = .5, annot = True, cmap = 'Blues',fmt=".0f")
plt.xlabel('Predicted label')
plt.ylabel('Actual label')
plt.suptitle('Confusion matrix',size = 15)
plt.title("Using C3D Approach - Max Pooling", loc='center')
plt.savefig("cm_max_c3d.png",dpi = 96)
plt.show()

In [ ]:
fpr_c3d_maxpool, tpr_c3d_maxpool, tr_c3d_max = roc_curve(Y_test_c3d, np.array(Y_pred_c3d_maxpool_round2))
auc_c3d_maxpool = roc_auc_score(Y_test_c3d, Y_pred_c3d_maxpool_round2)

plt.plot(fpr_c3d_maxpool, tpr_c3d_maxpool, 'b-o', label = 'AUC = %0.4f'%auc_c3d_maxpool)
plt.plot(fpr_c3d_maxpool,fpr_c3d_maxpool,linestyle = '--',color = 'k')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.suptitle('ROC curve',size = 15)
plt.title("Using C3D Approach - Max Pooling", loc='center')
plt.legend()
plt.savefig("roc_auc_max_c3d.png",dpi = 96)
plt.show()

# Using I3D Approach
# Step - 5: Designing the Deep Learning Model.
### Step - 5a: Design a model that performs Max Pooling.
### Step - 5b: After this pooling layer, stack a fuly connected layer with a dropout layer (having 60% probability) that predicts whether in a frame action is happening or not.
### Step - 5c : Compile the model and print the model summary.

In [ ]:
# Number of classes in the target variable
NB_CLASSES=1

# Create a sequencial model in Keras
model_max_i3d = tf.keras.models.Sequential(name = 'Deep_Neural_Network_Model_Max-Pool1D_I3D')

# Add a reshaping layer
model_max_i3d.add(tf.keras.layers.Reshape((1, 512),name = 'Reshaping'))
          
# Pooling using Max Pooling 1D:
model_max_i3d.add(keras.layers.MaxPool1D(pool_size = 2,strides = 1,padding = "same", name = 'Max-Pool1D-Layer'))

# Flattening the pooled features
model_max_i3d.add(keras.layers.Flatten(name = 'Flatten-Layer'))

# Add the first hidden layer
model_max_i3d.add(keras.layers.Dense(200,                    #Number of hidden nodes
                             kernel_regularizer = l2(0.01),
                             name='Hidden-Layer-1', #Logical name
                             activation='sigmoid'))    #activation function

# Add the second hidden layer
model_max_i3d.add(keras.layers.Dense(500,                    #Number of hidden nodes
                             name='Hidden-Layer-2', #Logical name
                             activation='sigmoid'))    #activation function

# Add the third hidden layer
model_max_i3d.add(keras.layers.Dense(200,                    #Number of hidden nodes
                             kernel_regularizer = l2(0.01),
                             name='Hidden-Layer-3', #Logical name
                             activation='sigmoid'))    #activation function

# Adding a Dropout layer of probability of 60%.
model_max_i3d.add(keras.layers.Dropout(0.6, name = 'Dropout-Layer'))

# Add an Output layer with sigmoid activation
model_max_i3d.add(keras.layers.Dense(NB_CLASSES,
                             name='Output-Layer',
                             activation='sigmoid'))

#Compile the model with loss & metrics
model_max_i3d.compile(optimizer = 'adam', loss = 'binary_crossentropy',
              metrics = ['accuracy'])

model_max_i3d.build(input_shape = (None,1,512))

#Print the model meta-data
model_max_i3d.summary()

In [ ]:
# Diagram of the Model Architecture
keras.utils.plot_model(model_max_i3d,to_file='model_max_i3d.png', show_shapes=True, show_layer_activations=True )

### Step - 5d: Train the model.

In [ ]:
#Make it verbose so we can see the progress
VERBOSE=1

#Set Batch size
BATCH_SIZE=120

#Set number of epochs
EPOCHS=25

print("\nTraining Progress:\n------------------------------------")

# Reshaping Train and Valid features so that it satisfies the fit function requirements. 
X_train_reshape_i3d = tf.reshape(X_train_i3d, [X_train_i3d.shape[0], 1, X_train_i3d.shape[1]])
X_valid_reshape_i3d = tf.reshape(X_valid_i3d, [X_valid_i3d.shape[0], 1, X_valid_i3d.shape[1]])

# Assigning weights to labels since there is an imbalanced in dataset labels.
# This will avoid overfitting as well as the model would not be bias.
weights_assigned = {0:1,1:892}

#Fit the model. This will perform the entire training cycle, including
#forward propagation, loss computation, backward propagation.
#Execute for the specified batch sizes and epoch
#Perform validation after each epoch.

history_max_i3d = model_max_i3d.fit(X_train_reshape_i3d,
          Y_train_i3d,
          class_weight = weights_assigned,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_data = (X_valid_reshape_i3d, Y_valid_i3d),
          verbose = VERBOSE)

### Step - 5e : Evaluate the model.

In [ ]:
#Evaluate the model against the test dataset and print results
print("\nEvaluation against Test Dataset (Using I3D Approach - Max Pooling) :\n------------------------------------")
X_test_reshape_i3d = tf.reshape(X_test_i3d, [X_test_i3d.shape[0], 1,X_test_i3d.shape[1]])
model_max_i3d.evaluate(X_test_reshape_i3d,Y_test_i3d)

### Step - 6: Plot train and validation accuracy and loss curves for the choosen epochs.

In [ ]:
plt.plot(history_max_i3d.history['accuracy'],'r-o', label = 'train')
plt.plot(history_max_i3d.history['val_accuracy'],'b-o', label = 'validation')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.suptitle("Train Accuracy vs Validation Accuracy")
plt.title("Using I3D Approach - Max Pooling", loc='center')
plt.legend()
plt.savefig("accuracy_max_i3d.png",dpi = 96)
plt.show()

In [ ]:
plt.plot(history_max_i3d.history['loss'],'r-o', label = 'train')
plt.plot(history_max_i3d.history['val_loss'],'g-o', label = 'validation')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.suptitle("Train Loss vs Validation Loss")
plt.title("Using I3D Approach - Max Pooling", loc='center')
plt.legend()
plt.savefig("loss_max_i3d.png",dpi = 96)
plt.show()

### Step - 7: Perform prediction of labels from the test features.

In [ ]:
Y_pred_i3d_maxpool = model_max_i3d.predict(X_test_i3d)
Y_pred_i3d_maxpool

In [ ]:
Y_pred_i3d_maxpool_round2 = Y_pred_i3d_maxpool.round(decimals=2)
Y_pred_i3d_maxpool_round2

In [ ]:
Y_pred_i3d_maxpool_round0 = Y_pred_i3d_maxpool.round(decimals=0)
Y_pred_i3d_maxpool_round0

### Step - 8: Calculating important metrics.
#### Confusion matrix, Classification report, Precision, Recall, F1 score, Accuracy.

In [ ]:
print("Using I3D Approach - Max Pooling")
print("Classification Report:")
print(classification_report(Y_test_i3d, Y_pred_i3d_maxpool_round0))
print("Confusion Matrix: ")
print(confusion_matrix(Y_test_i3d, Y_pred_i3d_maxpool_round0))
print('Precision Score : ',(precision_score(Y_test_i3d, Y_pred_i3d_maxpool_round0, average = 'macro')))
print('Recall Score : ',(recall_score(Y_test_i3d, Y_pred_i3d_maxpool_round0, average = 'macro')))
print('F-1 Score : ',(f1_score(Y_test_i3d, Y_pred_i3d_maxpool_round0, average = 'macro')))
print("Accuracy:", accuracy_score(Y_test_i3d, Y_pred_i3d_maxpool_round0))

### Step - 9: Plotting Confusion matrix, ROC curve, AUC score.

In [ ]:
cm_i3d_maxpool = confusion_matrix(Y_test_i3d, Y_pred_i3d_maxpool_round0)

plt.figure(figsize = (5,5))
sns.heatmap(data = cm_i3d_maxpool, linewidths = .5, annot = True, cmap = 'Blues',fmt=".0f")
plt.xlabel('Predicted label')
plt.ylabel('Actual label')
plt.suptitle('Confusion matrix',size = 15)
plt.title("Using I3D Approach - Max Pooling", loc='center')
plt.savefig("cm_max_i3d.png",dpi = 96)
plt.show()

In [ ]:
fpr_i3d_maxpool, tpr_i3d_maxpool, tr_i3d_max = roc_curve(Y_test_i3d, np.array(Y_pred_i3d_maxpool_round2))
auc_i3d_maxpool = roc_auc_score(Y_test_i3d, Y_pred_i3d_maxpool_round2)

plt.plot(fpr_i3d_maxpool, tpr_i3d_maxpool, 'b-o', label = 'AUC = %0.4f'%auc_i3d_maxpool)
plt.plot(fpr_i3d_maxpool,fpr_i3d_maxpool,linestyle = '--',color = 'k')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.suptitle('ROC curve',size = 15)
plt.title("Using I3D Approach - Max Pooling", loc='center')
plt.legend()
plt.savefig("roc_auc_max_i3d.png",dpi = 96)
plt.show()

# Using ResNET Approach
# Step - 5: Designing the Deep Learning Model.
### Step - 5a: Design a model that performs Average Pooling.
### Step - 5b: After this pooling layer, stack a fuly connected layer with a dropout layer (having 60% probability) that predicts whether in a frame action is happening or not.
### Step - 5c : Compile the model and print the model summary.

In [ ]:
# Number of classes in the target variable
NB_CLASSES=1

# Create a sequencial model in Keras
model_avg_resnet = tf.keras.models.Sequential(name = 'Deep_Neural_Network_Model_Average-Pool1D_ResNET')

# Add a reshaping layer
model_avg_resnet.add(tf.keras.layers.Reshape((1, 512),name = 'Reshaping'))
          
# Pooling using Max Pooling 1D:
model_avg_resnet.add(keras.layers.AveragePooling1D(pool_size = 2,strides = 1,padding = "same", name = 'Average-Pool1D-Layer'))

# Flattening the pooled features
model_avg_resnet.add(keras.layers.Flatten(name = 'Flatten-Layer'))

# Add the first hidden layer
model_avg_resnet.add(keras.layers.Dense(200,                    #Number of hidden nodes
                             kernel_regularizer = l2(0.01),
                             name='Hidden-Layer-1', #Logical name
                             activation='sigmoid'))    #activation function

# Add the second hidden layer
model_avg_resnet.add(keras.layers.Dense(500,                    #Number of hidden nodes
                             name='Hidden-Layer-2', #Logical name
                             activation='sigmoid'))    #activation function

# Add the third hidden layer
model_avg_resnet.add(keras.layers.Dense(200,                    #Number of hidden nodes
                             kernel_regularizer = l2(0.01),
                             name='Hidden-Layer-3', #Logical name
                             activation='sigmoid'))    #activation function

# Adding a Dropout layer of probability of 60%.
model_avg_resnet.add(keras.layers.Dropout(0.6, name = 'Dropout-Layer'))

# Add an Output layer with sigmoid activation
model_avg_resnet.add(keras.layers.Dense(NB_CLASSES,
                             name='Output-Layer',
                             activation='sigmoid'))

#Compile the model with loss & metrics
model_avg_resnet.compile(optimizer = 'adam', loss = 'binary_crossentropy',
              metrics = ['accuracy'])

model_avg_resnet.build(input_shape = (None,1,512))

#Print the model meta-data
model_avg_resnet.summary()

In [ ]:
# Diagram of the Model Architecture
keras.utils.plot_model(model_avg_resnet,to_file='model_avg_resnet.png', show_shapes=True, show_layer_activations=True )

### Step - 5d: Train the model.

In [ ]:
#Make it verbose so we can see the progress
VERBOSE=1

#Set Batch size
BATCH_SIZE=120

#Set number of epochs
EPOCHS=25

print("\nTraining Progress:\n------------------------------------")

# Reshaping Train and Valid features so that it satisfies the fit function requirements. 
X_train_reshape_resnet = tf.reshape(X_train_resnet, [X_train_resnet.shape[0], 1, X_train_resnet.shape[1]])
X_valid_reshape_resnet = tf.reshape(X_valid_resnet, [X_valid_resnet.shape[0], 1, X_valid_resnet.shape[1]])

# Assigning weights to labels since there is an imbalanced in dataset labels.
# This will avoid overfitting as well as the model would not be bias.
weights_assigned = {0:1,1:892}

#Fit the model. This will perform the entire training cycle, including
#forward propagation, loss computation, backward propagation.
#Execute for the specified batch sizes and epoch
#Perform validation after each epoch.

history_avg_resnet = model_avg_resnet.fit(X_train_reshape_resnet,
          Y_train_resnet,
          class_weight = weights_assigned,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_data = (X_valid_reshape_resnet, Y_valid_resnet),
          verbose = VERBOSE)

### Step - 5e : Evaluate the model.

In [ ]:
#Evaluate the model against the test dataset and print results
print("\nEvaluation against Test Dataset (Using ResNET Approach - Average Pooling) :\n------------------------------------")
X_test_reshape_resnet = tf.reshape(X_test_resnet, [X_test_resnet.shape[0], 1,X_test_resnet.shape[1]])
model_avg_resnet.evaluate(X_test_reshape_resnet,Y_test_resnet)

### Step - 6: Plot train and validation accuracy and loss curves for the choosen epochs.

In [ ]:
plt.plot(history_avg_resnet.history['accuracy'],'r-o', label = 'train')
plt.plot(history_avg_resnet.history['val_accuracy'],'b-o', label = 'validation')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.suptitle("Train Accuracy vs Validation Accuracy")
plt.title("Using ResNET Approach - Average Pooling", loc='center')
plt.legend()
plt.savefig("accuracy_avg_resnet.png",dpi = 96)
plt.show()

In [ ]:
plt.plot(history_avg_resnet.history['loss'],'r-o', label = 'train')
plt.plot(history_avg_resnet.history['val_loss'],'g-o', label = 'validation')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.suptitle("Train Loss vs Validation Loss")
plt.title("Using ResNET Approach - Average Pooling", loc='center')
plt.legend()
plt.savefig("loss_avg_resnet.png",dpi = 96)
plt.show()

### Step - 7: Perform prediction of labels from the test features.

In [ ]:
Y_pred_resnet_avgpool = model_avg_resnet.predict(X_test_resnet)
Y_pred_resnet_avgpool

In [ ]:
Y_pred_resnet_avgpool_round2 = Y_pred_resnet_avgpool.round(decimals=2)
Y_pred_resnet_avgpool_round2

In [ ]:
Y_pred_resnet_avgpool_round0 = Y_pred_resnet_avgpool.round(decimals=0)
Y_pred_resnet_avgpool_round0

### Step - 8: Calculating important metrics.
#### Confusion matrix, Classification report, Precision, Recall, F1 score, Accuracy.

In [ ]:
print("Using ResNET Approach - Average Pooling")
print("Classification Report:")
print(classification_report(Y_test_resnet, Y_pred_resnet_avgpool_round0))
print("Confusion Matrix: ")
print(confusion_matrix(Y_test_resnet, Y_pred_resnet_avgpool_round0))
print('Precision Score : ',(precision_score(Y_test_resnet, Y_pred_resnet_avgpool_round0, average = 'macro')))
print('Recall Score : ',(recall_score(Y_test_resnet, Y_pred_resnet_avgpool_round0, average = 'macro')))
print('F-1 Score : ',(f1_score(Y_test_resnet, Y_pred_resnet_avgpool_round0, average = 'macro')))
print("Accuracy:", accuracy_score(Y_test_resnet, Y_pred_resnet_avgpool_round0))

### Step - 9: Plotting Confusion matrix, ROC curve, AUC score.

In [ ]:
cm_resnet_avgpool = confusion_matrix(Y_test_resnet, Y_pred_resnet_avgpool_round0)

plt.figure(figsize = (5,5))
sns.heatmap(data = cm_resnet_avgpool, linewidths = .5, annot = True, cmap = 'Blues',fmt=".0f")
plt.xlabel('Predicted label')
plt.ylabel('Actual label')
plt.suptitle('Confusion matrix',size = 15)
plt.title("Using ResNET Approach - Average Pooling", loc='center')
plt.savefig("cm_avg_resnet.png",dpi = 96)
plt.show()

In [ ]:
fpr_resnet_avgpool, tpr_resnet_avgpool, tr_resnet_avg = roc_curve(Y_test_resnet, np.array(Y_pred_resnet_avgpool_round2))
auc_resnet_avgpool = roc_auc_score(Y_test_resnet, Y_pred_resnet_avgpool_round2)

plt.plot(fpr_resnet_avgpool, tpr_resnet_avgpool, 'b-o', label = 'AUC = %0.4f'%auc_resnet_avgpool)
plt.plot(fpr_resnet_avgpool,fpr_resnet_avgpool,linestyle = '--',color = 'k')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.suptitle('ROC curve',size = 15)
plt.title("Using ResNET Approach - Average Pooling", loc='center')
plt.legend()
plt.savefig("roc_auc_avg_resnet.png",dpi = 96)
plt.show()

# Using C3D Approach
# Step - 5: Designing the Deep Learning Model.
### Step - 5a: Design a model that performs Average Pooling.
### Step - 5b: After this pooling layer, stack a fuly connected layer with a dropout layer (having 60% probability) that predicts whether in a frame action is happening or not.
### Step - 5c : Compile the model and print the model summary.

In [ ]:
# Number of classes in the target variable
NB_CLASSES=1

# Create a sequencial model in Keras
model_avg_c3d = tf.keras.models.Sequential(name = 'Deep_Neural_Network_Model_Average-Pool1D_C3D')

# Add a reshaping layer
model_avg_c3d.add(tf.keras.layers.Reshape((1, 512),name = 'Reshaping'))
          
# Pooling using Max Pooling 1D:
model_avg_c3d.add(keras.layers.AveragePooling1D(pool_size = 2,strides = 1,padding = "same", name = 'Average-Pool1D-Layer'))

# Flattening the pooled features
model_avg_c3d.add(keras.layers.Flatten(name = 'Flatten-Layer'))

# Add the first hidden layer
model_avg_c3d.add(keras.layers.Dense(200,                    #Number of hidden nodes
                             kernel_regularizer = l2(0.01),
                             name='Hidden-Layer-1', #Logical name
                             activation='sigmoid'))    #activation function

# Add the second hidden layer
model_avg_c3d.add(keras.layers.Dense(500,                    #Number of hidden nodes
                             name='Hidden-Layer-2', #Logical name
                             activation='sigmoid'))    #activation function

# Add the third hidden layer
model_avg_c3d.add(keras.layers.Dense(200,                    #Number of hidden nodes
                             kernel_regularizer = l2(0.01),
                             name='Hidden-Layer-3', #Logical name
                             activation='sigmoid'))    #activation function

# Adding a Dropout layer of probability of 60%.
model_avg_c3d.add(keras.layers.Dropout(0.6, name = 'Dropout-Layer'))

# Add an Output layer with sigmoid activation
model_avg_c3d.add(keras.layers.Dense(NB_CLASSES,
                             name='Output-Layer',
                             activation='sigmoid'))

#Compile the model with loss & metrics
model_avg_c3d.compile(optimizer = 'adam', loss = 'binary_crossentropy',
              metrics = ['accuracy'])

model_avg_c3d.build(input_shape = (None,1,512))

#Print the model meta-data
model_avg_c3d.summary()

In [ ]:
# Diagram of the Model Architecture
keras.utils.plot_model(model_avg_c3d,to_file='model_avg_c3d.png', show_shapes=True, show_layer_activations=True )

### Step - 5d: Train the model.

In [ ]:
#Make it verbose so we can see the progress
VERBOSE=1

#Set Batch size
BATCH_SIZE=120

#Set number of epochs
EPOCHS=25

print("\nTraining Progress:\n------------------------------------")

# Reshaping Train and Valid features so that it satisfies the fit function requirements. 
X_train_reshape_c3d = tf.reshape(X_train_c3d, [X_train_c3d.shape[0], 1, X_train_c3d.shape[1]])
X_valid_reshape_c3d = tf.reshape(X_valid_c3d, [X_valid_c3d.shape[0], 1, X_valid_c3d.shape[1]])

# Assigning weights to labels since there is an imbalanced in dataset labels.
# This will avoid overfitting as well as the model would not be bias.
weights_assigned = {0:1,1:892}

#Fit the model. This will perform the entire training cycle, including
#forward propagation, loss computation, backward propagation.
#Execute for the specified batch sizes and epoch
#Perform validation after each epoch.

history_avg_c3d = model_avg_c3d.fit(X_train_reshape_c3d,
          Y_train_c3d,
          class_weight = weights_assigned,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_data = (X_valid_reshape_c3d, Y_valid_c3d),
          verbose = VERBOSE)

### Step - 5e : Evaluate the model.

In [ ]:
#Evaluate the model against the test dataset and print results
print("\nEvaluation against Test Dataset (Using C3D Approach - Average Pooling) :\n------------------------------------")
X_test_reshape_c3d = tf.reshape(X_test_c3d, [X_test_c3d.shape[0], 1,X_test_c3d.shape[1]])
model_avg_c3d.evaluate(X_test_reshape_c3d,Y_test_c3d)

### Step - 6: Plot train and validation accuracy and loss curves for the choosen epochs.

In [ ]:
plt.plot(history_avg_c3d.history['accuracy'],'r-o', label = 'train')
plt.plot(history_avg_c3d.history['val_accuracy'],'b-o', label = 'validation')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.suptitle("Train Accuracy vs Validation Accuracy")
plt.title("Using C3D Approach - Average Pooling", loc='center')
plt.legend()
plt.savefig("accuracy_avg_c3d.png",dpi = 96)
plt.show()

In [ ]:
plt.plot(history_avg_c3d.history['loss'],'r-o', label = 'train')
plt.plot(history_avg_c3d.history['val_loss'],'g-o', label = 'validation')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.suptitle("Train Loss vs Validation Loss")
plt.title("Using C3D Approach - Average Pooling", loc='center')
plt.legend()
plt.savefig("loss_avg_c3d.png",dpi = 96)
plt.show()

### Step - 7: Perform prediction of labels from the test features.

In [ ]:
Y_pred_c3d_avgpool = model_avg_c3d.predict(X_test_c3d)
Y_pred_c3d_avgpool

In [ ]:
Y_pred_c3d_avgpool_round2 = Y_pred_c3d_avgpool.round(decimals=2)
Y_pred_c3d_avgpool_round2

In [ ]:
Y_pred_c3d_avgpool_round0 = Y_pred_c3d_avgpool.round(decimals=0)
Y_pred_c3d_avgpool_round0

### Step - 8: Calculating important metrics.
#### Confusion matrix, Classification report, Precision, Recall, F1 score, Accuracy.

In [ ]:
print("Using C3D Approach - Average Pooling")
print("Classification Report:")
print(classification_report(Y_test_c3d, Y_pred_c3d_avgpool_round0))
print("Confusion Matrix: ")
print(confusion_matrix(Y_test_c3d, Y_pred_c3d_avgpool_round0))
print('Precision Score : ',(precision_score(Y_test_c3d, Y_pred_c3d_avgpool_round0, average = 'macro')))
print('Recall Score : ',(recall_score(Y_test_c3d, Y_pred_c3d_avgpool_round0, average = 'macro')))
print('F-1 Score : ',(f1_score(Y_test_c3d, Y_pred_c3d_avgpool_round0, average = 'macro')))
print("Accuracy:", accuracy_score(Y_test_c3d, Y_pred_c3d_avgpool_round0))

### Step - 9: Plotting Confusion matrix, ROC curve, AUC score.

In [ ]:
cm_c3d_avgpool = confusion_matrix(Y_test_c3d, Y_pred_c3d_avgpool_round0)

plt.figure(figsize = (5,5))
sns.heatmap(data = cm_c3d_avgpool, linewidths = .5, annot = True, cmap = 'Blues',fmt=".0f")
plt.xlabel('Predicted label')
plt.ylabel('Actual label')
plt.suptitle('Confusion matrix',size = 15)
plt.title("Using C3D Approach - Average Pooling", loc='center')
plt.savefig("cm_avg_c3d.png",dpi = 96)
plt.show()

In [ ]:
fpr_c3d_avgpool, tpr_c3d_avgpool, tr_c3d_avg = roc_curve(Y_test_c3d, np.array(Y_pred_c3d_avgpool_round2))
auc_c3d_avgpool = roc_auc_score(Y_test_c3d, Y_pred_c3d_avgpool_round2)

plt.plot(fpr_c3d_avgpool, tpr_c3d_avgpool, 'b-o', label = 'AUC = %0.4f'%auc_c3d_avgpool)
plt.plot(fpr_c3d_avgpool,fpr_c3d_avgpool,linestyle = '--',color = 'k')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.suptitle('ROC curve',size = 15)
plt.title("Using C3D Approach - Average Pooling", loc='center')
plt.legend()
plt.savefig("roc_auc_avg_c3d.png",dpi = 96)
plt.show()

# Using I3D Approach
# Step - 5: Designing the Deep Learning Model.
### Step - 5a: Design a model that performs Average Pooling.
### Step - 5b: After this pooling layer, stack a fuly connected layer with a dropout layer (having 60% probability) that predicts whether in a frame action is happening or not.
### Step - 5c : Compile the model and print the model summary.

In [ ]:
# Number of classes in the target variable
NB_CLASSES=1

# Create a sequencial model in Keras
model_avg_i3d = tf.keras.models.Sequential(name = 'Deep_Neural_Network_Model_Average-Pool1D_I3D')

# Add a reshaping layer
model_avg_i3d.add(tf.keras.layers.Reshape((1, 512),name = 'Reshaping'))
          
# Pooling using Max Pooling 1D:
model_avg_i3d.add(keras.layers.AveragePooling1D(pool_size = 2,strides = 1,padding = "same", name = 'Average-Pool1D-Layer'))

# Flattening the pooled features
model_avg_i3d.add(keras.layers.Flatten(name = 'Flatten-Layer'))

# Add the first hidden layer
model_avg_i3d.add(keras.layers.Dense(200,                    #Number of hidden nodes
                             kernel_regularizer = l2(0.01),
                             name='Hidden-Layer-1', #Logical name
                             activation='sigmoid'))    #activation function

# Add the second hidden layer
model_avg_i3d.add(keras.layers.Dense(500,                    #Number of hidden nodes
                             name='Hidden-Layer-2', #Logical name
                             activation='sigmoid'))    #activation function

# Add the third hidden layer
model_avg_i3d.add(keras.layers.Dense(200,                    #Number of hidden nodes
                             kernel_regularizer = l2(0.01),
                             name='Hidden-Layer-3', #Logical name
                             activation='sigmoid'))    #activation function

# Adding a Dropout layer of probability of 60%.
model_avg_i3d.add(keras.layers.Dropout(0.6, name = 'Dropout-Layer'))

# Add an Output layer with sigmoid activation
model_avg_i3d.add(keras.layers.Dense(NB_CLASSES,
                             name='Output-Layer',
                             activation='sigmoid'))

#Compile the model with loss & metrics
model_avg_i3d.compile(optimizer = 'adam', loss = 'binary_crossentropy',
              metrics = ['accuracy'])

model_avg_i3d.build(input_shape = (None,1,512))

#Print the model meta-data
model_avg_i3d.summary()

In [ ]:
# Diagram of the Model Architecture
keras.utils.plot_model(model_avg_i3d,to_file='model_avg_i3d.png', show_shapes=True, show_layer_activations=True )

### Step - 5d: Train the model.

In [ ]:
#Make it verbose so we can see the progress
VERBOSE=1

#Set Batch size
BATCH_SIZE=120

#Set number of epochs
EPOCHS=25

print("\nTraining Progress:\n------------------------------------")

# Reshaping Train and Valid features so that it satisfies the fit function requirements. 
X_train_reshape_i3d = tf.reshape(X_train_i3d, [X_train_i3d.shape[0], 1, X_train_i3d.shape[1]])
X_valid_reshape_i3d = tf.reshape(X_valid_i3d, [X_valid_i3d.shape[0], 1, X_valid_i3d.shape[1]])

# Assigning weights to labels since there is an imbalanced in dataset labels.
# This will avoid overfitting as well as the model would not be bias.
weights_assigned = {0:1,1:892}

#Fit the model. This will perform the entire training cycle, including
#forward propagation, loss computation, backward propagation.
#Execute for the specified batch sizes and epoch
#Perform validation after each epoch.

history_avg_i3d = model_avg_i3d.fit(X_train_reshape_i3d,
          Y_train_i3d,
          class_weight = weights_assigned,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_data = (X_valid_reshape_i3d, Y_valid_i3d),
          verbose = VERBOSE)

### Step - 5e : Evaluate the model.

In [ ]:
#Evaluate the model against the test dataset and print results
print("\nEvaluation against Test Dataset (Using I3D Approach - Average Pooling) :\n------------------------------------")
X_test_reshape_i3d = tf.reshape(X_test_i3d, [X_test_i3d.shape[0], 1,X_test_i3d.shape[1]])
model_avg_i3d.evaluate(X_test_reshape_i3d,Y_test_i3d)

### Step - 6: Plot train and validation accuracy and loss curves for the choosen epochs.

In [ ]:
plt.plot(history_avg_i3d.history['accuracy'],'r-o', label = 'train')
plt.plot(history_avg_i3d.history['val_accuracy'],'b-o', label = 'validation')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.suptitle("Train Accuracy vs Validation Accuracy")
plt.title("Using I3D Approach - Average Pooling", loc='center')
plt.legend()
plt.savefig("accuracy_avg_i3d.png",dpi = 96)
plt.show()

In [ ]:
plt.plot(history_avg_i3d.history['loss'],'r-o', label = 'train')
plt.plot(history_avg_i3d.history['val_loss'],'g-o', label = 'validation')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.suptitle("Train Loss vs Validation Loss")
plt.title("Using I3D Approach - Average Pooling", loc='center')
plt.legend()
plt.savefig("loss_avg_i3d.png",dpi = 96)
plt.show()

### Step - 7: Perform prediction of labels from the test features.

In [ ]:
Y_pred_i3d_avgpool = model_avg_i3d.predict(X_test_i3d)
Y_pred_i3d_avgpool

In [ ]:
Y_pred_i3d_avgpool_round2 = Y_pred_i3d_avgpool.round(decimals=2)
Y_pred_i3d_avgpool_round2

In [ ]:
Y_pred_i3d_avgpool_round0 = Y_pred_i3d_avgpool.round(decimals=0)
Y_pred_i3d_avgpool_round0

### Step - 8: Calculating important metrics.
#### Confusion matrix, Classification report, Precision, Recall, F1 score, Accuracy.

In [ ]:
print("Using I3D Approach - Average Pooling")
print("Classification Report:")
print(classification_report(Y_test_i3d, Y_pred_i3d_avgpool_round0))
print("Confusion Matrix: ")
print(confusion_matrix(Y_test_i3d, Y_pred_i3d_avgpool_round0))
print('Precision Score : ',(precision_score(Y_test_i3d, Y_pred_i3d_avgpool_round0, average = 'macro')))
print('Recall Score : ',(recall_score(Y_test_i3d, Y_pred_i3d_avgpool_round0, average = 'macro')))
print('F-1 Score : ',(f1_score(Y_test_i3d, Y_pred_i3d_avgpool_round0, average = 'macro')))
print("Accuracy:", accuracy_score(Y_test_i3d, Y_pred_i3d_avgpool_round0))

### Step - 9: Plotting Confusion matrix, ROC curve, AUC score.

In [ ]:
cm_i3d_avgpool = confusion_matrix(Y_test_i3d, Y_pred_i3d_avgpool_round0)

plt.figure(figsize = (5,5))
sns.heatmap(data = cm_i3d_avgpool, linewidths = .5, annot = True, cmap = 'Blues',fmt=".0f")
plt.xlabel('Predicted label')
plt.ylabel('Actual label')
plt.suptitle('Confusion matrix',size = 15)
plt.title("Using I3D Approach - Average Pooling", loc='center')
plt.savefig("cm_avg_i3d.png",dpi = 96)
plt.show()

In [ ]:
fpr_i3d_avgpool, tpr_i3d_avgpool, tr_i3d_avg = roc_curve(Y_test_i3d, np.array(Y_pred_i3d_avgpool_round2))
auc_i3d_avgpool = roc_auc_score(Y_test_i3d, Y_pred_i3d_avgpool_round2)

plt.plot(fpr_i3d_avgpool, tpr_i3d_avgpool, 'b-o', label = 'AUC = %0.4f'%auc_i3d_avgpool)
plt.plot(fpr_i3d_avgpool,fpr_i3d_avgpool,linestyle = '--',color = 'k')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.suptitle('ROC curve',size = 15)
plt.title("Using I3D Approach - Average Pooling", loc='center')
plt.legend()
plt.savefig("roc_auc_avg_i3d.png",dpi = 96)
plt.show()

In [ ]:
# Define mixed max-average pooling layer
class MixedPooling(tf.keras.layers.Layer):
    def __init__(self, alpha, size=2, name = 'Mixed-Pool1D-Layer'):
        super(MixedPooling, self).__init__(name = name)
        self.alpha = self.add_weight(shape = (),dtype=tf.float32, initializer=tf.keras.initializers.Constant(alpha))
        self.max_pool = tf.keras.layers.MaxPooling1D(pool_size=size, strides=1, padding='same')
        self.avg_pool = tf.keras.layers.AveragePooling1D(pool_size=size, strides=1, padding='same')

    def call(self, inputs):
        x1 = self.max_pool(inputs)
        x2 = self.avg_pool(inputs)
        outputs = tf.add(tf.multiply(x1, self.alpha), tf.multiply(x2, (1-self.alpha)))
        return outputs

# Using ResNET Approach
# Step - 5: Designing the Deep Learning Model.
### Step - 5a: Design a model that performs Mixed Pooling.
### Step - 5b: After this pooling layer, stack a fuly connected layer with a dropout layer (having 60% probability) that predicts whether in a frame action is happening or not.
### Step - 5c : Compile the model and print the model summary.

In [ ]:
# Number of classes in the target variable
NB_CLASSES=1

# Create a sequencial model in Keras
model_mix_resnet = tf.keras.models.Sequential(name = 'Deep_Neural_Network_Model_Mixed-Pool1D_ResNET')

# Add a reshaping layer
model_mix_resnet.add(tf.keras.layers.Reshape((1, 512),name = 'Reshaping'))
          
# Pooling using Mixed Pooling (MaxPool1D and AveragePool1D)
model_mix_resnet.add(MixedPooling(alpha=0.7, size=2))

# Flattening the pooled features
model_mix_resnet.add(keras.layers.Flatten(name = 'Flatten-Layer'))

# Add the first hidden layer
model_mix_resnet.add(keras.layers.Dense(200,                    #Number of hidden nodes
                             kernel_regularizer = l2(0.01),
                             name='Hidden-Layer-1', #Logical name
                             activation='sigmoid'))    #activation function

# Add the second hidden layer
model_mix_resnet.add(keras.layers.Dense(500,                    #Number of hidden nodes
                             name='Hidden-Layer-2', #Logical name
                             activation='sigmoid'))    #activation function

# Add the third hidden layer
model_mix_resnet.add(keras.layers.Dense(200,                    #Number of hidden nodes
                             kernel_regularizer = l2(0.01),
                             name='Hidden-Layer-3', #Logical name
                             activation='sigmoid'))    #activation function

# Adding a Dropout layer of probability of 60%.
model_mix_resnet.add(keras.layers.Dropout(0.6, name = 'Dropout-Layer'))

# Add an Output layer with sigmoid activation
model_mix_resnet.add(keras.layers.Dense(NB_CLASSES,
                             name='Output-Layer',
                             activation='sigmoid'))

#Compile the model with loss & metrics
model_mix_resnet.compile(optimizer = 'adam', loss = 'binary_crossentropy',
              metrics = ['accuracy'])

model_mix_resnet.build(input_shape = (None,1,512))

#Print the model meta-data
model_mix_resnet.summary()

In [ ]:
# Diagram of the Model Architecture
keras.utils.plot_model(model_mix_resnet,to_file='model_mix_resnet.png', show_shapes=True, show_layer_activations=True )

### Step - 5d: Train the model.

In [ ]:
#Make it verbose so we can see the progress
VERBOSE=1

#Set Batch size
BATCH_SIZE=120

#Set number of epochs
EPOCHS=25

print("\nTraining Progress:\n------------------------------------")

# Reshaping Train and Valid features so that it satisfies the fit function requirements. 
X_train_reshape_resnet = tf.reshape(X_train_resnet, [X_train_resnet.shape[0], 1, X_train_resnet.shape[1]])
X_valid_reshape_resnet = tf.reshape(X_valid_resnet, [X_valid_resnet.shape[0], 1, X_valid_resnet.shape[1]])

# Assigning weights to labels since there is an imbalanced in dataset labels.
# This will avoid overfitting as well as the model would not be bias.
weights_assigned = {0:1,1:892}

#Fit the model. This will perform the entire training cycle, including
#forward propagation, loss computation, backward propagation.
#Execute for the specified batch sizes and epoch
#Perform validation after each epoch.

history_mix_resnet = model_mix_resnet.fit(X_train_reshape_resnet,
          Y_train_resnet,
          class_weight = weights_assigned,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_data = (X_valid_reshape_resnet, Y_valid_resnet),
          verbose = VERBOSE)

### Step - 5e : Evaluate the model.

In [ ]:
#Evaluate the model against the test dataset and print results
print("\nEvaluation against Test Dataset (Using ResNET Approach - Mixed Pooling) :\n------------------------------------")
X_test_reshape_resnet = tf.reshape(X_test_resnet, [X_test_resnet.shape[0], 1,X_test_resnet.shape[1]])
model_mix_resnet.evaluate(X_test_reshape_resnet,Y_test_resnet)

### Step - 6: Plot train and validation accuracy and loss curves for the choosen epochs.

In [ ]:
plt.plot(history_mix_resnet.history['accuracy'],'r-o', label = 'train')
plt.plot(history_mix_resnet.history['val_accuracy'],'b-o', label = 'validation')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.suptitle("Train Accuracy vs Validation Accuracy")
plt.title("Using ResNET Approach - Mixed Pooling", loc='center')
plt.legend()
plt.savefig("accuracy_mix_resnet.png",dpi = 96)
plt.show()

In [ ]:
plt.plot(history_mix_resnet.history['loss'],'r-o', label = 'train')
plt.plot(history_mix_resnet.history['val_loss'],'g-o', label = 'validation')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.suptitle("Train Loss vs Validation Loss")
plt.title("Using ResNET Approach - Mixed Pooling", loc='center')
plt.legend()
plt.savefig("loss_mix_resnet.png",dpi = 96)
plt.show()

### Step - 7: Perform prediction of labels from the test features.

In [ ]:
Y_pred_resnet_mixpool = model_mix_resnet.predict(X_test_resnet)
Y_pred_resnet_mixpool

In [ ]:
Y_pred_resnet_mixpool_round2 = Y_pred_resnet_mixpool.round(decimals=2)
Y_pred_resnet_mixpool_round2

In [ ]:
Y_pred_resnet_mixpool_round0 = Y_pred_resnet_mixpool.round(decimals=0)
Y_pred_resnet_mixpool_round0

### Step - 8: Calculating important metrics.
#### Confusion matrix, Classification report, Precision, Recall, F1 score, Accuracy.

In [ ]:
print("Using ResNET Approach - Mixed Pooling")
print("Classification Report:")
print(classification_report(Y_test_resnet, Y_pred_resnet_mixpool_round0))
print("Confusion Matrix: ")
print(confusion_matrix(Y_test_resnet, Y_pred_resnet_mixpool_round0))
print('Precision Score : ',(precision_score(Y_test_resnet, Y_pred_resnet_mixpool_round0, average = 'macro')))
print('Recall Score : ',(recall_score(Y_test_resnet, Y_pred_resnet_mixpool_round0, average = 'macro')))
print('F-1 Score : ',(f1_score(Y_test_resnet, Y_pred_resnet_mixpool_round0, average = 'macro')))
print("Accuracy:", accuracy_score(Y_test_resnet, Y_pred_resnet_mixpool_round0))

### Step - 9: Plotting Confusion matrix, ROC curve, AUC score.

In [ ]:
cm_resnet_mixpool = confusion_matrix(Y_test_resnet, Y_pred_resnet_mixpool_round0)

plt.figure(figsize = (5,5))
sns.heatmap(data = cm_resnet_mixpool, linewidths = .5, annot = True, cmap = 'Blues',fmt=".0f")
plt.xlabel('Predicted label')
plt.ylabel('Actual label')
plt.suptitle('Confusion matrix',size = 15)
plt.title("Using ResNET Approach - Mixed Pooling", loc='center')
plt.savefig("cm_mix_resnet.png",dpi = 96)
plt.show()

In [ ]:
fpr_resnet_mixpool, tpr_resnet_mixpool, tr_resnet_mix = roc_curve(Y_test_resnet, np.array(Y_pred_resnet_mixpool_round2))
auc_resnet_mixpool = roc_auc_score(Y_test_resnet, Y_pred_resnet_mixpool_round2)

plt.plot(fpr_resnet_mixpool, tpr_resnet_mixpool, 'b-o', label = 'AUC = %0.4f'%auc_resnet_mixpool)
plt.plot(fpr_resnet_mixpool,fpr_resnet_mixpool,linestyle = '--',color = 'k')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.suptitle('ROC curve',size = 15)
plt.title("Using ResNET Approach - Mixed Pooling", loc='center')
plt.legend()
plt.savefig("roc_auc_mix_resnet.png",dpi = 96)
plt.show()

# Using C3D Approach
# Step - 5: Designing the Deep Learning Model.
### Step - 5a: Design a model that performs Mixed Pooling.
### Step - 5b: After this pooling layer, stack a fuly connected layer with a dropout layer (having 60% probability) that predicts whether in a frame action is happening or not.
### Step - 5c : Compile the model and print the model summary.

In [ ]:
# Number of classes in the target variable
NB_CLASSES=1

# Create a sequencial model in Keras
model_mix_c3d = tf.keras.models.Sequential(name = 'Deep_Neural_Network_Model_Mixed-Pool1D_C3D')

# Add a reshaping layer
model_mix_c3d.add(tf.keras.layers.Reshape((1, 512),name = 'Reshaping'))
          
# Pooling using Mixed Pooling (MaxPool1D and AveragePool1D)
model_mix_c3d.add(MixedPooling(alpha=0.7, size=2))

# Flattening the pooled features
model_mix_c3d.add(keras.layers.Flatten(name = 'Flatten-Layer'))

# Add the first hidden layer
model_mix_c3d.add(keras.layers.Dense(200,                    #Number of hidden nodes
                             kernel_regularizer = l2(0.01),
                             name='Hidden-Layer-1', #Logical name
                             activation='sigmoid'))    #activation function

# Add the second hidden layer
model_mix_c3d.add(keras.layers.Dense(500,                    #Number of hidden nodes
                             name='Hidden-Layer-2', #Logical name
                             activation='sigmoid'))    #activation function

# Add the third hidden layer
model_mix_c3d.add(keras.layers.Dense(200,                    #Number of hidden nodes
                             kernel_regularizer = l2(0.01),
                             name='Hidden-Layer-3', #Logical name
                             activation='sigmoid'))    #activation function

# Adding a Dropout layer of probability of 60%.
model_mix_c3d.add(keras.layers.Dropout(0.6, name = 'Dropout-Layer'))

# Add an Output layer with sigmoid activation
model_mix_c3d.add(keras.layers.Dense(NB_CLASSES,
                             name='Output-Layer',
                             activation='sigmoid'))

#Compile the model with loss & metrics
model_mix_c3d.compile(optimizer = 'adam', loss = 'binary_crossentropy',
              metrics = ['accuracy'])

model_mix_c3d.build(input_shape = (None,1,512))

#Print the model meta-data
model_mix_c3d.summary()

In [ ]:
# Diagram of the Model Architecture
keras.utils.plot_model(model_mix_c3d,to_file='model_mix_c3d.png', show_shapes=True, show_layer_activations=True )

### Step - 5d: Train the model.

In [ ]:
#Make it verbose so we can see the progress
VERBOSE=1

#Set Batch size
BATCH_SIZE=120

#Set number of epochs
EPOCHS=25

print("\nTraining Progress:\n------------------------------------")

# Reshaping Train and Valid features so that it satisfies the fit function requirements. 
X_train_reshape_c3d = tf.reshape(X_train_c3d, [X_train_c3d.shape[0], 1, X_train_c3d.shape[1]])
X_valid_reshape_c3d = tf.reshape(X_valid_c3d, [X_valid_c3d.shape[0], 1, X_valid_c3d.shape[1]])

# Assigning weights to labels since there is an imbalanced in dataset labels.
# This will avoid overfitting as well as the model would not be bias.
weights_assigned = {0:1,1:892}

#Fit the model. This will perform the entire training cycle, including
#forward propagation, loss computation, backward propagation.
#Execute for the specified batch sizes and epoch
#Perform validation after each epoch.

history_mix_c3d = model_mix_c3d.fit(X_train_reshape_c3d,
          Y_train_c3d,
          class_weight = weights_assigned,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_data = (X_valid_reshape_c3d, Y_valid_c3d),
          verbose = VERBOSE)

### Step - 5e : Evaluate the model.

In [ ]:
#Evaluate the model against the test dataset and print results
print("\nEvaluation against Test Dataset (Using C3D Approach - Mixed Pooling) :\n------------------------------------")
X_test_reshape_c3d = tf.reshape(X_test_c3d, [X_test_c3d.shape[0], 1,X_test_c3d.shape[1]])
model_mix_c3d.evaluate(X_test_reshape_c3d,Y_test_c3d)

### Step - 6: Plot train and validation accuracy and loss curves for the choosen epochs.

In [ ]:
plt.plot(history_mix_c3d.history['accuracy'],'r-o', label = 'train')
plt.plot(history_mix_c3d.history['val_accuracy'],'b-o', label = 'validation')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.suptitle("Train Accuracy vs Validation Accuracy")
plt.title("Using C3D Approach - Mixed Pooling", loc='center')
plt.legend()
plt.savefig("accuracy_mix_c3d.png",dpi = 96)
plt.show()

In [ ]:
plt.plot(history_mix_c3d.history['loss'],'r-o', label = 'train')
plt.plot(history_mix_c3d.history['val_loss'],'g-o', label = 'validation')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.suptitle("Train Loss vs Validation Loss")
plt.title("Using C3D Approach - Mixed Pooling", loc='center')
plt.legend()
plt.savefig("loss_mix_c3d.png",dpi = 96)
plt.show()

### Step - 7: Perform prediction of labels from the test features.

In [ ]:
Y_pred_c3d_mixpool = model_mix_c3d.predict(X_test_c3d)
Y_pred_c3d_mixpool

In [ ]:
Y_pred_c3d_mixpool_round2 = Y_pred_c3d_mixpool.round(decimals=2)
Y_pred_c3d_mixpool_round2

In [ ]:
Y_pred_c3d_mixpool_round0 = Y_pred_c3d_mixpool.round(decimals=0)
Y_pred_c3d_mixpool_round0

### Step - 8: Calculating important metrics.
#### Confusion matrix, Classification report, Precision, Recall, F1 score, Accuracy.

In [ ]:
print("Using C3D Approach - Mixed Pooling")
print("Classification Report:")
print(classification_report(Y_test_c3d, Y_pred_c3d_mixpool_round0))
print("Confusion Matrix: ")
print(confusion_matrix(Y_test_c3d, Y_pred_c3d_mixpool_round0))
print('Precision Score : ',(precision_score(Y_test_c3d, Y_pred_c3d_mixpool_round0, average = 'macro')))
print('Recall Score : ',(recall_score(Y_test_c3d, Y_pred_c3d_mixpool_round0, average = 'macro')))
print('F-1 Score : ',(f1_score(Y_test_c3d, Y_pred_c3d_mixpool_round0, average = 'macro')))
print("Accuracy:", accuracy_score(Y_test_c3d, Y_pred_c3d_mixpool_round0))

### Step - 9: Plotting Confusion matrix, ROC curve, AUC score.

In [ ]:
cm_c3d_mixpool = confusion_matrix(Y_test_c3d, Y_pred_c3d_mixpool_round0)

plt.figure(figsize = (5,5))
sns.heatmap(data = cm_c3d_mixpool, linewidths = .5, annot = True, cmap = 'Blues',fmt=".0f")
plt.xlabel('Predicted label')
plt.ylabel('Actual label')
plt.suptitle('Confusion matrix',size = 15)
plt.title("Using C3D Approach - Mixed Pooling", loc='center')
plt.savefig("cm_mix_c3d.png",dpi = 96)
plt.show()

In [ ]:
fpr_c3d_mixpool, tpr_c3d_mixpool, tr_c3d_mix = roc_curve(Y_test_c3d, np.array(Y_pred_c3d_mixpool_round2))
auc_c3d_mixpool = roc_auc_score(Y_test_c3d, Y_pred_c3d_mixpool_round2)

plt.plot(fpr_c3d_mixpool, tpr_c3d_mixpool, 'b-o', label = 'AUC = %0.4f'%auc_c3d_mixpool)
plt.plot(fpr_c3d_mixpool,fpr_c3d_mixpool,linestyle = '--',color = 'k')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.suptitle('ROC curve',size = 15)
plt.title("Using C3D Approach - Mixed Pooling", loc='center')
plt.legend()
plt.savefig("roc_auc_mix_c3d.png",dpi = 96)
plt.show()

# Using I3D Approach
# Step - 5: Designing the Deep Learning Model.
### Step - 5a: Design a model that performs Mixed Pooling.
### Step - 5b: After this pooling layer, stack a fuly connected layer with a dropout layer (having 60% probability) that predicts whether in a frame action is happening or not.
### Step - 5c : Compile the model and print the model summary.

In [ ]:
# Number of classes in the target variable
NB_CLASSES=1

# Create a sequencial model in Keras
model_mix_i3d = tf.keras.models.Sequential(name = 'Deep_Neural_Network_Model_Mixed-Pool1D_I3D')

# Add a reshaping layer
model_mix_i3d.add(tf.keras.layers.Reshape((1, 512),name = 'Reshaping'))
          
# Pooling using Mixed Pooling (MaxPool1D and AveragePool1D)
model_mix_i3d.add(MixedPooling(alpha=0.7, size=2))

# Flattening the pooled features
model_mix_i3d.add(keras.layers.Flatten(name = 'Flatten-Layer'))

# Add the first hidden layer
model_mix_i3d.add(keras.layers.Dense(200,                    #Number of hidden nodes
                             kernel_regularizer = l2(0.01),
                             name='Hidden-Layer-1', #Logical name
                             activation='sigmoid'))    #activation function

# Add the second hidden layer
model_mix_i3d.add(keras.layers.Dense(500,                    #Number of hidden nodes
                             name='Hidden-Layer-2', #Logical name
                             activation='sigmoid'))    #activation function

# Add the third hidden layer
model_mix_i3d.add(keras.layers.Dense(200,                    #Number of hidden nodes
                             kernel_regularizer = l2(0.01),
                             name='Hidden-Layer-3', #Logical name
                             activation='sigmoid'))    #activation function

# Adding a Dropout layer of probability of 60%.
model_mix_i3d.add(keras.layers.Dropout(0.6, name = 'Dropout-Layer'))

# Add an Output layer with sigmoid activation
model_mix_i3d.add(keras.layers.Dense(NB_CLASSES,
                             name='Output-Layer',
                             activation='sigmoid'))

#Compile the model with loss & metrics
model_mix_i3d.compile(optimizer = 'adam', loss = 'binary_crossentropy',
              metrics = ['accuracy'])

model_mix_i3d.build(input_shape = (None,1,512))

#Print the model meta-data
model_mix_i3d.summary()

In [ ]:
# Diagram of the Model Architecture
keras.utils.plot_model(model_mix_i3d,to_file='model_mix_i3d.png', show_shapes=True, show_layer_activations=True )

### Step - 5d: Train the model.

In [ ]:
#Make it verbose so we can see the progress
VERBOSE=1

#Set Batch size
BATCH_SIZE=120

#Set number of epochs
EPOCHS=25

print("\nTraining Progress:\n------------------------------------")

# Reshaping Train and Valid features so that it satisfies the fit function requirements. 
X_train_reshape_i3d = tf.reshape(X_train_i3d, [X_train_i3d.shape[0], 1, X_train_i3d.shape[1]])
X_valid_reshape_i3d = tf.reshape(X_valid_i3d, [X_valid_i3d.shape[0], 1, X_valid_i3d.shape[1]])

# Assigning weights to labels since there is an imbalanced in dataset labels.
# This will avoid overfitting as well as the model would not be bias.
weights_assigned = {0:1,1:892}

#Fit the model. This will perform the entire training cycle, including
#forward propagation, loss computation, backward propagation.
#Execute for the specified batch sizes and epoch
#Perform validation after each epoch.

history_mix_i3d = model_mix_i3d.fit(X_train_reshape_i3d,
          Y_train_i3d,
          class_weight = weights_assigned,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_data = (X_valid_reshape_i3d, Y_valid_i3d),
          verbose = VERBOSE)

### Step - 5e : Evaluate the model.

In [ ]:
#Evaluate the model against the test dataset and print results
print("\nEvaluation against Test Dataset (Using I3D Approach - Mixed Pooling) :\n------------------------------------")
X_test_reshape_i3d = tf.reshape(X_test_i3d, [X_test_i3d.shape[0], 1,X_test_i3d.shape[1]])
model_mix_i3d.evaluate(X_test_reshape_i3d,Y_test_i3d)

### Step - 6: Plot train and validation accuracy and loss curves for the choosen epochs.

In [ ]:
plt.plot(history_mix_i3d.history['accuracy'],'r-o', label = 'train')
plt.plot(history_mix_i3d.history['val_accuracy'],'b-o', label = 'validation')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.suptitle("Train Accuracy vs Validation Accuracy")
plt.title("Using I3D Approach - Mixed Pooling", loc='center')
plt.legend()
plt.savefig("accuracy_mix_i3d.png",dpi = 96)
plt.show()

In [ ]:
plt.plot(history_mix_i3d.history['loss'],'r-o', label = 'train')
plt.plot(history_mix_i3d.history['val_loss'],'g-o', label = 'validation')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.suptitle("Train Loss vs Validation Loss")
plt.title("Using I3D Approach - Mixed Pooling", loc='center')
plt.legend()
plt.savefig("loss_mix_i3d.png",dpi = 96)
plt.show()

### Step - 7: Perform prediction of labels from the test features.

In [ ]:
Y_pred_i3d_mixpool = model_mix_i3d.predict(X_test_i3d)
Y_pred_i3d_mixpool

In [ ]:
Y_pred_i3d_mixpool_round2 = Y_pred_i3d_mixpool.round(decimals=2)
Y_pred_i3d_mixpool_round2

In [ ]:
Y_pred_i3d_mixpool_round0 = Y_pred_i3d_mixpool.round(decimals=0)
Y_pred_i3d_mixpool_round0

### Step - 8: Calculating important metrics.
#### Confusion matrix, Classification report, Precision, Recall, F1 score, Accuracy.

In [ ]:
print("Using I3D Approach - Mixed Pooling")
print("Classification Report:")
print(classification_report(Y_test_i3d, Y_pred_i3d_mixpool_round0))
print("Confusion Matrix: ")
print(confusion_matrix(Y_test_i3d, Y_pred_i3d_mixpool_round0))
print('Precision Score : ',(precision_score(Y_test_i3d, Y_pred_i3d_mixpool_round0, average = 'macro')))
print('Recall Score : ',(recall_score(Y_test_i3d, Y_pred_i3d_mixpool_round0, average = 'macro')))
print('F-1 Score : ',(f1_score(Y_test_i3d, Y_pred_i3d_mixpool_round0, average = 'macro')))
print("Accuracy:", accuracy_score(Y_test_i3d, Y_pred_i3d_mixpool_round0))

### Step - 9: Plotting Confusion matrix, ROC curve, AUC score.

In [ ]:
cm_i3d_mixpool = confusion_matrix(Y_test_i3d, Y_pred_i3d_mixpool_round0)

plt.figure(figsize = (5,5))
sns.heatmap(data = cm_i3d_mixpool, linewidths = .5, annot = True, cmap = 'Blues',fmt=".0f")
plt.xlabel('Predicted label')
plt.ylabel('Actual label')
plt.suptitle('Confusion matrix',size = 15)
plt.title("Using I3D Approach - Mixed Pooling", loc='center')
plt.savefig("cm_mix_i3d.png",dpi = 96)
plt.show()

In [ ]:
fpr_i3d_mixpool, tpr_i3d_mixpool, tr_i3d_mix = roc_curve(Y_test_i3d, np.array(Y_pred_i3d_mixpool_round2))
auc_i3d_mixpool = roc_auc_score(Y_test_i3d, Y_pred_i3d_mixpool_round2)

plt.plot(fpr_i3d_mixpool, tpr_i3d_mixpool, 'b-o', label = 'AUC = %0.4f'%auc_i3d_mixpool)
plt.plot(fpr_i3d_mixpool,fpr_i3d_mixpool,linestyle = '--',color = 'k')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.suptitle('ROC curve',size = 15)
plt.title("Using I3D Approach - Mixed Pooling", loc='center')
plt.legend()
plt.savefig("roc_auc_mix_i3d.png",dpi = 96)
plt.show()

In [ ]:
# Define NetVLAD Pooling

import tensorflow as tf
import math
import numpy as np

class PoolingBaseModel:
    """Inherit from this class when implementing new models."""

    def __init__(self, feature_size, max_samples, cluster_size, output_dim,
                 gating=True, add_batch_norm=True, is_training=True):
        """Initialize a NetVLAD block.

        Args:
            feature_size: Dimensionality of the input features.
            max_samples: The maximum number of samples to pool.
            cluster_size: The number of clusters.
            output_dim: size of the output space after dimension reduction.
            add_batch_norm: (bool) if True, adds batch normalization.
            is_training: (bool) Whether or not the graph is training.
        """

        self.feature_size = feature_size
        self.max_samples = max_samples
        self.output_dim = output_dim
        self.is_training = is_training
        self.gating = gating
        self.add_batch_norm = add_batch_norm
        self.cluster_size = cluster_size

    def forward(self, reshaped_input):
        raise NotImplementedError("Models should implement the forward pass.")
        
    def context_gating(self, input_layer):
        """Context Gating

        Args:
            input_layer: Input layer in the following shape:
            'batch_size' x 'number_of_activation'

        Returns:
            activation: gated layer in the following shape:
            'batch_size' x 'number_of_activation'
        """

        input_dim = input_layer.shape[1]

        gating_weights = tf.Variable(name="gating_weights",
                                     shape=[input_dim, input_dim],
                                     initializer=tf.random_normal_initializer(
                                         stddev=1 / math.sqrt(input_dim)))

        gates = tf.matmul(input_layer, gating_weights)

        if self.add_batch_norm:
            gates = layers.BatchNormalization()(gates, training=self.is_training, name="gating_bn")
        else:
            gating_biases = tf.Variable(name="gating_biases",
                                        shape=[input_dim],
                                        initializer=tf.random_normal_initializer(
                                            stddev=1 / math.sqrt(input_dim)))
            gates += gating_biases

        gates = tf.sigmoid(gates)

        activation = tf.multiply(input_layer, gates)

        return activation    
    

#class NetVLAD(PoolingBaseModel):
class NetVLAD(tf.keras.Model):
    """Creates a NetVLAD class.
    """
    def __init__(self, feature_size, max_samples, cluster_size, output_dim,
                 gating=True, add_batch_norm=True, is_training=True):
        #super(PoolingBaseModel, self).__init__()
        super(NetVLAD, self).__init__()
        self.feature_size = feature_size
        self.max_samples = max_samples
        self.cluster_size = cluster_size
        self.output_dim = output_dim
        self.gating = gating
        self.add_batch_norm = add_batch_norm
        self.is_training = is_training

        self.cluster_weights = tf.Variable(tf.random.normal(
            shape=[self.feature_size, self.cluster_size],
            stddev=1 / math.sqrt(self.feature_size)),
            name="cluster_weights")

        if not add_batch_norm:
            self.cluster_biases = tf.Variable(tf.random.normal(
                shape=[self.cluster_size],
                stddev=1 / math.sqrt(self.feature_size)),
                name="cluster_biases")

        self.cluster_weights2 = tf.Variable(tf.random.normal(
            shape=[1, self.feature_size, self.cluster_size],
            stddev=1 / math.sqrt(self.feature_size)),
            name="cluster_weights2")

        self.hidden1_weights = tf.Variable(tf.random.normal(
            shape=[self.cluster_size * self.feature_size, self.output_dim],
            stddev=1 / math.sqrt(self.cluster_size)),
            name="hidden1_weights")
    
    def call(self, inputs):
        """Forward pass of a NetVLAD block.

        Args:
        inputs: If your input is in that form:
        'batch_size' x 'max_samples' x 'feature_size'
        It should be reshaped in the following form:
        'batch_size*max_samples' x 'feature_size'
        by performing:
        inputs = tf.reshape(input, [-1, feature_size])

        Returns:
        vlad: the pooled vector of size: 'batch_size' x 'output_dim'
        """
        reshaped_input = tf.reshape(inputs, [-1, self.feature_size])

        activation = tf.matmul(reshaped_input, self.cluster_weights)

        if self.add_batch_norm:
            activation = tf.keras.layers.BatchNormalization(
                name="cluster_bn")(activation, training=self.is_training)
        else:
            activation += self.cluster_biases

        activation = tf.nn.softmax(activation)

        activation = tf.reshape(activation,
                                [-1, self.max_samples, self.cluster_size])

        a_sum = tf.reduce_sum(activation, axis=-2, keepdims=True)

        a = tf.multiply(a_sum, self.cluster_weights2)

        activation = tf.transpose(activation, perm=[0, 2, 1])

        reshaped_input = tf.reshape(reshaped_input, [-1,
                                                     self.max_samples, self.feature_size])

        vlad = tf.matmul(activation, reshaped_input)
        vlad = tf.transpose(vlad, perm=[0, 2, 1])
        vlad = tf.subtract(vlad, a)

        vlad = tf.nn.l2_normalize(vlad, axis=1)

        vlad = tf.reshape(vlad, [-1, self.cluster_size * self.feature_size])
        vlad = tf.nn.l2_normalize(vlad, axis=1)

        vlad = tf.matmul(vlad, self.hidden1_weights)

        if self.gating:
            vlad = PoolingBaseModel.context_gating(vlad)

        return vlad

# Using ResNET Approach
# Step - 5: Designing the Deep Learning Model.
### Step - 5a: Design a model that performs NetVLAD Pooling.
### Step - 5b: After this pooling layer, stack a fuly connected layer with a dropout layer (having 60% probability) that predicts whether in a frame action is happening or not.
### Step - 5c : Compile the model and print the model summary.

In [ ]:
# Number of classes in the target variable
NB_CLASSES=1

# Create a sequencial model in Keras
model_netvlad_resnet = tf.keras.models.Sequential(name = 'Deep_Neural_Network_Model_NetVLAD_Pool_ResNET')

# Add a reshaping layer
model_netvlad_resnet.add(tf.keras.layers.Reshape((1, 512),name = 'Reshaping'))
          
# Pooling using NetVLAD: (without gating, batch normalization, is_training)
model_netvlad_resnet.add(NetVLAD(feature_size=512, max_samples=1, cluster_size=64, output_dim=512, gating = False, add_batch_norm = False, is_training = False))

# Flattening the pooled features
model_netvlad_resnet.add(keras.layers.Flatten(name = 'Flatten-Layer'))

# Add the first hidden layer
model_netvlad_resnet.add(keras.layers.Dense(200,                    #Number of hidden nodes
                             kernel_regularizer = l2(0.01),
                             name='Hidden-Layer-1', #Logical name
                             activation='sigmoid'))    #activation function

# Add the second hidden layer
model_netvlad_resnet.add(keras.layers.Dense(500,                    #Number of hidden nodes
                             name='Hidden-Layer-2', #Logical name
                             activation='sigmoid'))    #activation function

# Add the third hidden layer
model_netvlad_resnet.add(keras.layers.Dense(200,                    #Number of hidden nodes
                             kernel_regularizer = l2(0.01),
                             name='Hidden-Layer-3', #Logical name
                             activation='sigmoid'))    #activation function

# Adding a Dropout layer of probability of 60%.
model_netvlad_resnet.add(keras.layers.Dropout(0.6, name = 'Dropout-Layer'))

# Add an Output layer with sigmoid activation
model_netvlad_resnet.add(keras.layers.Dense(NB_CLASSES,
                             name='Output-Layer',
                             activation='sigmoid'))

#Compile the model with loss & metrics
model_netvlad_resnet.compile(optimizer = 'adam', loss = 'binary_crossentropy',
              metrics = ['accuracy'])

model_netvlad_resnet.build(input_shape = (None,1,512))

#Print the model meta-data
model_netvlad_resnet.summary()

In [ ]:
# Diagram of the Model Architecture
keras.utils.plot_model(model_netvlad_resnet,to_file='model_netvlad_resnet.png', show_shapes=True, show_layer_activations=True )

### Step - 5d: Train the model.

In [ ]:
#Make it verbose so we can see the progress
VERBOSE=1

#Set Batch size
BATCH_SIZE=120

#Set number of epochs
EPOCHS=25

print("\nTraining Progress:\n------------------------------------")

# Reshaping Train and Valid features so that it satisfies the fit function requirements. 
X_train_reshape_resnet = tf.reshape(X_train_resnet, [X_train_resnet.shape[0], 1, X_train_resnet.shape[1]])
X_valid_reshape_resnet = tf.reshape(X_valid_resnet, [X_valid_resnet.shape[0], 1, X_valid_resnet.shape[1]])

# Assigning weights to labels since there is an imbalanced in dataset labels.
# This will avoid overfitting as well as the model would not be bias.
weights_assigned = {0:1,1:892}

#Fit the model. This will perform the entire training cycle, including
#forward propagation, loss computation, backward propagation.
#Execute for the specified batch sizes and epoch
#Perform validation after each epoch.

history_netvlad_resnet = model_netvlad_resnet.fit(X_train_reshape_resnet,
          Y_train_resnet,
          class_weight = weights_assigned,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_data = (X_valid_reshape_resnet, Y_valid_resnet),
          verbose = VERBOSE)

### Step - 5e : Evaluate the model.

In [ ]:
#Evaluate the model against the test dataset and print results
print("\nEvaluation against Test Dataset (Using ResNET Approach - NetVLAD Pooling) :\n------------------------------------")
X_test_reshape_resnet = tf.reshape(X_test_resnet, [X_test_resnet.shape[0], 1,X_test_resnet.shape[1]])
model_netvlad_resnet.evaluate(X_test_reshape_resnet,Y_test_resnet)

### Step - 6: Plot train and validation accuracy and loss curves for the choosen epochs.

In [ ]:
plt.plot(history_netvlad_resnet.history['accuracy'],'r-o', label = 'train')
plt.plot(history_netvlad_resnet.history['val_accuracy'],'b-o', label = 'validation')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.suptitle("Train Accuracy vs Validation Accuracy")
plt.title("Using ResNET Approach - NetVLAD Pooling", loc='center')
plt.legend()
plt.savefig("accuracy_netvlad_resnet.png",dpi = 96)
plt.show()

In [ ]:
plt.plot(history_netvlad_resnet.history['loss'],'r-o', label = 'train')
plt.plot(history_netvlad_resnet.history['val_loss'],'g-o', label = 'validation')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.suptitle("Train Loss vs Validation Loss")
plt.title("Using ResNET Approach - NetVLAD Pooling", loc='center')
plt.legend()
plt.savefig("loss_netvlad_resnet.png",dpi = 96)
plt.show()

### Step - 7: Perform prediction of labels from the test features.

In [ ]:
Y_pred_resnet_netvlad = model_netvlad_resnet.predict(X_test_resnet)
Y_pred_resnet_netvlad

In [ ]:
Y_pred_resnet_netvlad_round2 = Y_pred_resnet_netvlad.round(decimals=2)
Y_pred_resnet_netvlad_round2

In [ ]:
Y_pred_resnet_netvlad_round0 = Y_pred_resnet_netvlad.round(decimals=0)
Y_pred_resnet_netvlad_round0

### Step - 8: Calculating important metrics.
#### Confusion matrix, Classification report, Precision, Recall, F1 score, Accuracy.

In [ ]:
print("Using ResNET Approach - NetVLAD Pooling")
print("Classification Report:")
print(classification_report(Y_test_resnet, Y_pred_resnet_netvlad_round0))
print("Confusion Matrix: ")
print(confusion_matrix(Y_test_resnet, Y_pred_resnet_netvlad_round0))
print('Precision Score : ',(precision_score(Y_test_resnet, Y_pred_resnet_netvlad_round0, average = 'macro')))
print('Recall Score : ',(recall_score(Y_test_resnet, Y_pred_resnet_netvlad_round0, average = 'macro')))
print('F-1 Score : ',(f1_score(Y_test_resnet, Y_pred_resnet_netvlad_round0, average = 'macro')))
print("Accuracy:", accuracy_score(Y_test_resnet, Y_pred_resnet_netvlad_round0))

### Step - 9: Plotting Confusion matrix, ROC curve, AUC score.

In [ ]:
cm_resnet_netvlad = confusion_matrix(Y_test_resnet, Y_pred_resnet_netvlad_round0)

plt.figure(figsize = (5,5))
sns.heatmap(data = cm_resnet_netvlad, linewidths = .5, annot = True, cmap = 'Blues',fmt=".0f")
plt.xlabel('Predicted label')
plt.ylabel('Actual label')
plt.suptitle('Confusion matrix',size = 15)
plt.title("Using ResNET Approach - NetVLAD Pooling", loc='center')
plt.savefig("cm_netvlad_resnet.png",dpi = 96)
plt.show()

In [ ]:
fpr_resnet_netvlad, tpr_resnet_netvlad, tr_resnet_netvlad = roc_curve(Y_test_resnet, np.array(Y_pred_resnet_netvlad_round2))
auc_resnet_netvlad = roc_auc_score(Y_test_resnet, Y_pred_resnet_netvlad_round2)

plt.plot(fpr_resnet_netvlad, tpr_resnet_netvlad, 'b-o', label = 'AUC = %0.4f'%auc_resnet_netvlad)
plt.plot(fpr_resnet_netvlad,fpr_resnet_netvlad,linestyle = '--',color = 'k')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.suptitle('ROC curve',size = 15)
plt.title("Using ResNET Approach - NetVLAD Pooling", loc='center')
plt.legend()
plt.savefig("roc_auc_netvlad_resnet.png",dpi = 96)
plt.show()

# Using C3D Approach
# Step - 5: Designing the Deep Learning Model.
### Step - 5a: Design a model that performs NetVLAD Pooling.
### Step - 5b: After this pooling layer, stack a fuly connected layer with a dropout layer (having 60% probability) that predicts whether in a frame action is happening or not.
### Step - 5c : Compile the model and print the model summary.

In [ ]:
# Number of classes in the target variable
NB_CLASSES=1

# Create a sequencial model in Keras
model_netvlad_c3d = tf.keras.models.Sequential(name = 'Deep_Neural_Network_Model_NetVLAD_Pool_C3D')

# Add a reshaping layer
model_netvlad_c3d.add(tf.keras.layers.Reshape((1, 512),name = 'Reshaping'))
          
# Pooling using NetVLAD: (without gating, batch normalization, is_training)
model_netvlad_c3d.add(NetVLAD(feature_size=512, max_samples=1, cluster_size=64, output_dim=512, gating = False, add_batch_norm = False, is_training = False))

# Flattening the pooled features
model_netvlad_c3d.add(keras.layers.Flatten(name = 'Flatten-Layer'))

# Add the first hidden layer
model_netvlad_c3d.add(keras.layers.Dense(200,                    #Number of hidden nodes
                             kernel_regularizer = l2(0.01),
                             name='Hidden-Layer-1', #Logical name
                             activation='sigmoid'))    #activation function

# Add the second hidden layer
model_netvlad_c3d.add(keras.layers.Dense(500,                    #Number of hidden nodes
                             name='Hidden-Layer-2', #Logical name
                             activation='sigmoid'))    #activation function

# Add the third hidden layer
model_netvlad_c3d.add(keras.layers.Dense(200,                    #Number of hidden nodes
                             kernel_regularizer = l2(0.01),
                             name='Hidden-Layer-3', #Logical name
                             activation='sigmoid'))    #activation function

# Adding a Dropout layer of probability of 60%.
model_netvlad_c3d.add(keras.layers.Dropout(0.6, name = 'Dropout-Layer'))

# Add an Output layer with sigmoid activation
model_netvlad_c3d.add(keras.layers.Dense(NB_CLASSES,
                             name='Output-Layer',
                             activation='sigmoid'))

#Compile the model with loss & metrics
model_netvlad_c3d.compile(optimizer = 'adam', loss = 'binary_crossentropy',
              metrics = ['accuracy'])

model_netvlad_c3d.build(input_shape = (None,1,512))

#Print the model meta-data
model_netvlad_c3d.summary()

In [ ]:
# Diagram of the Model Architecture
keras.utils.plot_model(model_netvlad_c3d,to_file='model_netvlad_c3d.png', show_shapes=True, show_layer_activations=True )

### Step - 5d: Train the model.

In [ ]:
#Make it verbose so we can see the progress
VERBOSE=1

#Set Batch size
BATCH_SIZE=120

#Set number of epochs
EPOCHS=25

print("\nTraining Progress:\n------------------------------------")

# Reshaping Train and Valid features so that it satisfies the fit function requirements. 
X_train_reshape_c3d = tf.reshape(X_train_c3d, [X_train_c3d.shape[0], 1, X_train_c3d.shape[1]])
X_valid_reshape_c3d = tf.reshape(X_valid_c3d, [X_valid_c3d.shape[0], 1, X_valid_c3d.shape[1]])

# Assigning weights to labels since there is an imbalanced in dataset labels.
# This will avoid overfitting as well as the model would not be bias.
weights_assigned = {0:1,1:892}

#Fit the model. This will perform the entire training cycle, including
#forward propagation, loss computation, backward propagation.
#Execute for the specified batch sizes and epoch
#Perform validation after each epoch.

history_netvlad_c3d = model_netvlad_c3d.fit(X_train_reshape_c3d,
          Y_train_c3d,
          class_weight = weights_assigned,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_data = (X_valid_reshape_c3d, Y_valid_c3d),
          verbose = VERBOSE)

### Step - 5e : Evaluate the model.

In [ ]:
#Evaluate the model against the test dataset and print results
print("\nEvaluation against Test Dataset (Using C3D Approach - NetVLAD Pooling) :\n------------------------------------")
X_test_reshape_c3d = tf.reshape(X_test_c3d, [X_test_c3d.shape[0], 1,X_test_c3d.shape[1]])
model_netvlad_c3d.evaluate(X_test_reshape_c3d,Y_test_c3d)

### Step - 6: Plot train and validation accuracy and loss curves for the choosen epochs.

In [ ]:
plt.plot(history_netvlad_c3d.history['accuracy'],'r-o', label = 'train')
plt.plot(history_netvlad_c3d.history['val_accuracy'],'b-o', label = 'validation')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.suptitle("Train Accuracy vs Validation Accuracy")
plt.title("Using C3D Approach - NetVLAD Pooling", loc='center')
plt.legend()
plt.savefig("accuracy_netvlad_c3d.png",dpi = 96)
plt.show()

In [ ]:
plt.plot(history_netvlad_c3d.history['loss'],'r-o', label = 'train')
plt.plot(history_netvlad_c3d.history['val_loss'],'g-o', label = 'validation')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.suptitle("Train Loss vs Validation Loss")
plt.title("Using C3D Approach - NetVLAD Pooling", loc='center')
plt.legend()
plt.savefig("loss_netvlad_c3d.png",dpi = 96)
plt.show()

### Step - 7: Perform prediction of labels from the test features.

In [ ]:
Y_pred_c3d_netvlad = model_netvlad_c3d.predict(X_test_c3d)
Y_pred_c3d_netvlad

In [ ]:
Y_pred_c3d_netvlad_round2 = Y_pred_c3d_netvlad.round(decimals=2)
Y_pred_c3d_netvlad_round2

In [ ]:
Y_pred_c3d_netvlad_round0 = Y_pred_c3d_netvlad.round(decimals=0)
Y_pred_c3d_netvlad_round0

### Step - 8: Calculating important metrics.
#### Confusion matrix, Classification report, Precision, Recall, F1 score, Accuracy.

In [ ]:
print("Using C3D Approach - NetVLAD Pooling")
print("Classification Report:")
print(classification_report(Y_test_c3d, Y_pred_c3d_netvlad_round0))
print("Confusion Matrix: ")
print(confusion_matrix(Y_test_c3d, Y_pred_c3d_netvlad_round0))
print('Precision Score : ',(precision_score(Y_test_c3d, Y_pred_c3d_netvlad_round0, average = 'macro')))
print('Recall Score : ',(recall_score(Y_test_c3d, Y_pred_c3d_netvlad_round0, average = 'macro')))
print('F-1 Score : ',(f1_score(Y_test_c3d, Y_pred_c3d_netvlad_round0, average = 'macro')))
print("Accuracy:", accuracy_score(Y_test_c3d, Y_pred_c3d_netvlad_round0))

### Step - 9: Plotting Confusion matrix, ROC curve, AUC score.

In [ ]:
cm_c3d_netvlad = confusion_matrix(Y_test_c3d, Y_pred_c3d_netvlad_round0)

plt.figure(figsize = (5,5))
sns.heatmap(data = cm_c3d_netvlad, linewidths = .5, annot = True, cmap = 'Blues',fmt=".0f")
plt.xlabel('Predicted label')
plt.ylabel('Actual label')
plt.suptitle('Confusion matrix',size = 15)
plt.title("Using C3D Approach - NetVLAD Pooling", loc='center')
plt.savefig("cm_netvlad_c3d.png",dpi = 96)
plt.show()

In [ ]:
fpr_c3d_netvlad, tpr_c3d_netvlad, tr_c3d_netvlad = roc_curve(Y_test_c3d, np.array(Y_pred_c3d_netvlad_round2))
auc_c3d_netvlad = roc_auc_score(Y_test_c3d, Y_pred_c3d_netvlad_round2)

plt.plot(fpr_c3d_netvlad, tpr_c3d_netvlad, 'b-o', label = 'AUC = %0.4f'%auc_c3d_netvlad)
plt.plot(fpr_c3d_netvlad,fpr_c3d_netvlad,linestyle = '--',color = 'k')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.suptitle('ROC curve',size = 15)
plt.title("Using C3D Approach - NetVLAD Pooling", loc='center')
plt.legend()
plt.savefig("roc_auc_netvlad_c3d.png",dpi = 96)
plt.show()

# Using I3D Approach
# Step - 5: Designing the Deep Learning Model.
### Step - 5a: Design a model that performs NetVLAD Pooling.
### Step - 5b: After this pooling layer, stack a fuly connected layer with a dropout layer (having 60% probability) that predicts whether in a frame action is happening or not.
### Step - 5c : Compile the model and print the model summary.

In [ ]:
# Number of classes in the target variable
NB_CLASSES=1

# Create a sequencial model in Keras
model_netvlad_i3d = tf.keras.models.Sequential(name = 'Deep_Neural_Network_Model_NetVLAD_Pool_I3D')

# Add a reshaping layer
model_netvlad_i3d.add(tf.keras.layers.Reshape((1, 512),name = 'Reshaping'))
          
# Pooling using NetVLAD: (without gating, batch normalization, is_training)
model_netvlad_i3d.add(NetVLAD(feature_size=512, max_samples=1, cluster_size=64, output_dim=512, gating = False, add_batch_norm = False, is_training = False))

# Flattening the pooled features
model_netvlad_i3d.add(keras.layers.Flatten(name = 'Flatten-Layer'))

# Add the first hidden layer
model_netvlad_i3d.add(keras.layers.Dense(200,                    #Number of hidden nodes
                             kernel_regularizer = l2(0.01),
                             name='Hidden-Layer-1', #Logical name
                             activation='sigmoid'))    #activation function

# Add the second hidden layer
model_netvlad_i3d.add(keras.layers.Dense(500,                    #Number of hidden nodes
                             name='Hidden-Layer-2', #Logical name
                             activation='sigmoid'))    #activation function

# Add the third hidden layer
model_netvlad_i3d.add(keras.layers.Dense(200,                    #Number of hidden nodes
                             kernel_regularizer = l2(0.01),
                             name='Hidden-Layer-3', #Logical name
                             activation='sigmoid'))    #activation function

# Adding a Dropout layer of probability of 60%.
model_netvlad_i3d.add(keras.layers.Dropout(0.6, name = 'Dropout-Layer'))

# Add an Output layer with sigmoid activation
model_netvlad_i3d.add(keras.layers.Dense(NB_CLASSES,
                             name='Output-Layer',
                             activation='sigmoid'))

#Compile the model with loss & metrics
model_netvlad_i3d.compile(optimizer = 'adam', loss = 'binary_crossentropy',
              metrics = ['accuracy'])

model_netvlad_i3d.build(input_shape = (None,1,512))

#Print the model meta-data
model_netvlad_i3d.summary()

In [ ]:
# Diagram of the Model Architecture
keras.utils.plot_model(model_netvlad_i3d,to_file='model_netvlad_i3d.png', show_shapes=True, show_layer_activations=True )

### Step - 5d: Train the model.

In [ ]:
#Make it verbose so we can see the progress
VERBOSE=1

#Set Batch size
BATCH_SIZE=120

#Set number of epochs
EPOCHS=25

print("\nTraining Progress:\n------------------------------------")

# Reshaping Train and Valid features so that it satisfies the fit function requirements. 
X_train_reshape_i3d = tf.reshape(X_train_i3d, [X_train_i3d.shape[0], 1, X_train_i3d.shape[1]])
X_valid_reshape_i3d = tf.reshape(X_valid_i3d, [X_valid_i3d.shape[0], 1, X_valid_i3d.shape[1]])

# Assigning weights to labels since there is an imbalanced in dataset labels.
# This will avoid overfitting as well as the model would not be bias.
weights_assigned = {0:1,1:892}

#Fit the model. This will perform the entire training cycle, including
#forward propagation, loss computation, backward propagation.
#Execute for the specified batch sizes and epoch
#Perform validation after each epoch.

history_netvlad_i3d = model_netvlad_i3d.fit(X_train_reshape_i3d,
          Y_train_i3d,
          class_weight = weights_assigned,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_data = (X_valid_reshape_i3d, Y_valid_i3d),
          verbose = VERBOSE)

### Step - 5e : Evaluate the model.

In [ ]:
#Evaluate the model against the test dataset and print results
print("\nEvaluation against Test Dataset (Using I3D Approach - NetVLAD Pooling) :\n------------------------------------")
X_test_reshape_i3d = tf.reshape(X_test_i3d, [X_test_i3d.shape[0], 1,X_test_i3d.shape[1]])
model_netvlad_i3d.evaluate(X_test_reshape_i3d,Y_test_i3d)

### Step - 6: Plot train and validation accuracy and loss curves for the choosen epochs.

In [ ]:
plt.plot(history_netvlad_i3d.history['accuracy'],'r-o', label = 'train')
plt.plot(history_netvlad_i3d.history['val_accuracy'],'b-o', label = 'validation')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.suptitle("Train Accuracy vs Validation Accuracy")
plt.title("Using I3D Approach - NetVLAD Pooling", loc='center')
plt.legend()
plt.savefig("accuracy_netvlad_i3d.png",dpi = 96)
plt.show()

In [ ]:
plt.plot(history_netvlad_i3d.history['loss'],'r-o', label = 'train')
plt.plot(history_netvlad_i3d.history['val_loss'],'g-o', label = 'validation')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.suptitle("Train Loss vs Validation Loss")
plt.title("Using I3D Approach - NetVLAD Pooling", loc='center')
plt.legend()
plt.savefig("loss_netvlad_i3d.png",dpi = 96)
plt.show()

### Step - 7: Perform prediction of labels from the test features.

In [ ]:
Y_pred_i3d_netvlad = model_netvlad_i3d.predict(X_test_i3d)
Y_pred_i3d_netvlad

In [ ]:
Y_pred_i3d_netvlad_round2 = Y_pred_i3d_netvlad.round(decimals=2)
Y_pred_i3d_netvlad_round2

In [ ]:
Y_pred_i3d_netvlad_round0 = Y_pred_i3d_netvlad.round(decimals=0)
Y_pred_i3d_netvlad_round0

### Step - 8: Calculating important metrics.
#### Confusion matrix, Classification report, Precision, Recall, F1 score, Accuracy.

In [ ]:
print("Using I3D Approach - NetVLAD Pooling")
print("Classification Report:")
print(classification_report(Y_test_i3d, Y_pred_i3d_netvlad_round0))
print("Confusion Matrix: ")
print(confusion_matrix(Y_test_i3d, Y_pred_i3d_netvlad_round0))
print('Precision Score : ',(precision_score(Y_test_i3d, Y_pred_i3d_netvlad_round0, average = 'macro')))
print('Recall Score : ',(recall_score(Y_test_i3d, Y_pred_i3d_netvlad_round0, average = 'macro')))
print('F-1 Score : ',(f1_score(Y_test_i3d, Y_pred_i3d_netvlad_round0, average = 'macro')))
print("Accuracy:", accuracy_score(Y_test_i3d, Y_pred_i3d_netvlad_round0))

### Step - 9: Plotting Confusion matrix, ROC curve, AUC score.

In [ ]:
cm_i3d_netvlad = confusion_matrix(Y_test_i3d, Y_pred_i3d_netvlad_round0)

plt.figure(figsize = (5,5))
sns.heatmap(data = cm_i3d_netvlad, linewidths = .5, annot = True, cmap = 'Blues',fmt=".0f")
plt.xlabel('Predicted label')
plt.ylabel('Actual label')
plt.suptitle('Confusion matrix',size = 15)
plt.title("Using I3D Approach - NetVLAD Pooling", loc='center')
plt.savefig("cm_netvlad_i3d.png",dpi = 96)
plt.show()

In [ ]:
fpr_i3d_netvlad, tpr_i3d_netvlad, tr_i3d_netvlad = roc_curve(Y_test_i3d, np.array(Y_pred_i3d_netvlad_round2))
auc_i3d_netvlad = roc_auc_score(Y_test_i3d, Y_pred_i3d_netvlad_round2)

plt.plot(fpr_i3d_netvlad, tpr_i3d_netvlad, 'b-o', label = 'AUC = %0.4f'%auc_i3d_netvlad)
plt.plot(fpr_i3d_netvlad,fpr_i3d_netvlad,linestyle = '--',color = 'k')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.suptitle('ROC curve',size = 15)
plt.title("Using I3D Approach - NetVLAD Pooling", loc='center')
plt.legend()
plt.savefig("roc_auc_netvlad_i3d.png",dpi = 96)
plt.show()